In [143]:
#General
import numpy as np
import pandas as pd

#Data Retrieval and Visualization
import pandas_datareader.data as web
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

#Statistical Package
from scipy import stats 
import statsmodels.api as sm
import math
from scipy.stats import linregress
from scipy.stats import f

In [144]:
from pandas_datareader.famafrench import get_available_datasets
#spy_etf=web.DataReader('SPY','yahoo','2010-01-01','2020-01-01' ) 
beg1='1963-07-01' #'1963-07-01' '2015-01-01'
end1='2014-12-01' #'2014-12-01''2020-09-01'

In [145]:
def fm_summary(p): #Summary Statistics Function
    s = p.describe().T
    s['std_error'] = s['std']/np.sqrt(s['count'])
    s['tstat'] = s['mean']/s['std_error']
    return s[['mean','std','tstat']].T
def update_withrf(dataport):
    for x in range(len(dataport.columns)):
        dataport.iloc[:,x]=dataport.iloc[:,x]-dataport['RF']
    dataport.drop('RF',axis=1,inplace=True)
        

In [146]:
len(get_available_datasets())
#print(ds['DESCR'])
datasets = get_available_datasets()
datasets

['F-F_Research_Data_Factors',
 'F-F_Research_Data_Factors_weekly',
 'F-F_Research_Data_Factors_daily',
 'F-F_Research_Data_5_Factors_2x3',
 'F-F_Research_Data_5_Factors_2x3_daily',
 'Portfolios_Formed_on_ME',
 'Portfolios_Formed_on_ME_Wout_Div',
 'Portfolios_Formed_on_ME_Daily',
 'Portfolios_Formed_on_BE-ME',
 'Portfolios_Formed_on_BE-ME_Wout_Div',
 'Portfolios_Formed_on_BE-ME_Daily',
 'Portfolios_Formed_on_OP',
 'Portfolios_Formed_on_OP_Wout_Div',
 'Portfolios_Formed_on_OP_Daily',
 'Portfolios_Formed_on_INV',
 'Portfolios_Formed_on_INV_Wout_Div',
 'Portfolios_Formed_on_INV_Daily',
 '6_Portfolios_2x3',
 '6_Portfolios_2x3_Wout_Div',
 '6_Portfolios_2x3_weekly',
 '6_Portfolios_2x3_daily',
 '25_Portfolios_5x5',
 '25_Portfolios_5x5_Wout_Div',
 '25_Portfolios_5x5_Daily',
 '100_Portfolios_10x10',
 '100_Portfolios_10x10_Wout_Div',
 '100_Portfolios_10x10_Daily',
 '6_Portfolios_ME_OP_2x3',
 '6_Portfolios_ME_OP_2x3_Wout_Div',
 '6_Portfolios_ME_OP_2x3_daily',
 '25_Portfolios_ME_OP_5x5',
 '25_Portf

In [147]:
#Extracting data
betaPort=web.DataReader('25_Portfolios_ME_BETA_5x5','famafrench',start=beg1,end=end1)
niPort=web.DataReader('25_Portfolios_ME_NI_5x5','famafrench',start=beg1,end=end1)
varPort=web.DataReader('25_Portfolios_ME_VAR_5x5','famafrench',start=beg1,end=end1)
rVarPort=web.DataReader('25_Portfolios_ME_RESVAR_5x5','famafrench',start=beg1,end=end1)
acPort=web.DataReader('25_Portfolios_ME_AC_5x5','famafrench',start=beg1,end=end1)
momPort=web.DataReader('25_Portfolios_ME_Prior_12_2','famafrench',start=beg1,end=end1)
FF5=web.DataReader('F-F_Research_Data_5_Factors_2x3','famafrench',start=beg1,end=end1)
MOM=web.DataReader('F-F_Momentum_Factor','famafrench',start=beg1,end=end1)


In [148]:
betaPort

{0:          SMALL LoBETA  ME1 BETA2  ME1 BETA3  ME1 BETA4  SMALL HiBETA  \
 Date                                                                   
 1963-07       -0.6156    -0.5950     0.1464    -0.4538       -1.0076   
 1963-08        1.2546     4.0686     3.6955     2.0609        3.2983   
 1963-09       -0.8082    -2.0279     0.0034    -1.5411       -3.2312   
 1963-10       -0.2628     1.5439     0.2554     4.2260        2.5346   
 1963-11        0.6858    -2.6122    -0.5701    -1.4607       -1.6136   
 ...               ...        ...        ...        ...           ...   
 2014-08        3.6847     4.0526     4.4137     2.9171        3.6693   
 2014-09       -3.6194    -5.0385    -6.1376    -6.8540       -9.8558   
 2014-10        5.9265     7.5963     6.8120     5.1069        1.9413   
 2014-11       -0.9760    -1.5549    -0.6512    -0.8763       -1.0257   
 2014-12        5.8356     4.0965     5.3248     3.0467        3.3079   
 
          ME2 BETA1  ME2 BETA2  ME2 BETA3  ME2

In [149]:
FF5[0]
FF6=pd.concat([FF5[0],MOM[0]],axis=1,sort=False)

In [150]:
FF6.columns

Index(['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'Mom   '], dtype='object')

## Table 1

In [151]:
Table1=fm_summary(FF6).copy()
Table1.rename(str.upper,axis='columns',inplace=True)
Table1.drop('RF',axis=1).applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)

,MKT-RF,SMB,HML,RMW,CMA,MOM
mean,0.508,0.264,0.378,0.267,0.322,0.687
std,4.457,3.048,2.819,2.232,2.019,4.220
tstat,2.831,2.154,3.331,2.975,3.960,4.048


## Table 2 

In [152]:
#Excess return of each anomaly portfolio
betaPortXs = betaPort[0].sub(FF5[0]['RF'],axis=0)
niPortXs = niPort[0].sub(FF5[0]['RF'],axis=0)
varPortXs = varPort[0].sub(FF5[0]['RF'],axis=0)
rVarPortXs = rVarPort[0].sub(FF5[0]['RF'],axis=0)
acPortXs = acPort[0].sub(FF5[0]['RF'],axis=0)
momPortXs = momPort[0].sub(FF5[0]['RF'],axis=0)


In [153]:
def GRS(R,F): #GRS Statistics
    R=R.astype('float64')
    F=F.astype('float64')
    alphas = []
    alpha2=[]
    alphasquared=[]
    errors = []
    betas=[]
    returns=[]
    risquared=[]
    rsquared=[]
    bsealphas=[]
    for p in R.columns:
        model = sm.OLS(R[:][p], sm.add_constant(F[:]))
        fit = model.fit()
        alphas.append(fit.params["const"])
        alpha2.append(abs(fit.params["const"]))
        alphasquared.append(abs(fit.params["const"])**2)
        risquared.append((R[:][p].mean()-F['Mkt-RF'].mean())**2)
        betas.append(fit.params[1:])
        errors.append(fit.resid)
        returns.append(abs(R[:][p].mean()-F['Mkt-RF'].mean()))
        
        rsquared.append(fit.rsquared_adj)
        bsealphas.append(fit.bse["const"]**2)
   #For remaining statistics to compute mean. Convert list to arrasy
    alpha2=pd.np.array(alpha2)
    alphasquared=pd.np.array(alphasquared)
    returns=pd.np.array(returns)
    rsquared=pd.np.array(rsquared)
    bsealphas=pd.np.array(bsealphas)
    risquared=pd.np.array(risquared)
    #For GRS
    alphas = pd.np.array(alphas)
    errors = pd.np.array(errors).transpose()
    Ri=returns.mean()
    K = F.shape[1]
    T, N = errors.shape

    Ve = errors.transpose().dot(errors) / (T - K - 1)#unbiased estimate of the covariance matrix of residuals

    r = F.mean().values #step 5Factor mean vector

    f = F.values

    Vf = (f-r).transpose().dot(f-r) / (T - 1) #Step 7 unbiased estimate of the covariance matrix of the factors

    Sh2_a = alphas.dot(pd.np.linalg.pinv(Ve)).dot(alphas)
    Sh2_f = r.dot(pd.np.linalg.pinv(Vf)).dot(r)
    Sh2_a, Sh2_f
    ans=(T/N)*((T-N-K)/(T-K-1))*(Sh2_a/(1+Sh2_f))
    ans=ans.astype(float)
    #print("P-value of the GRS statistics: ",(1-f.cdf(ans,N,T-N-K)))
    #print("Average Absolute value of intercept A|ai|: ",alpha2.mean())
    #print("Average Absolute value of intercept/return A|ai|/A|ri| : ",alpha2.mean()/Ri)
    #print("Average Absolute value of intercept/return squared: ",(alpha2.mean()**2/Ri**2))
    #print("Average Absolute value of the As^2(ai)/A(ai)^2: ",(bsealphas.mean()))
    #print("Average value of the regression R^2 corrected for degress of freedom: ",rsquared.mean())
    return (ans,N,T-N-K,alpha2.mean(),alpha2.mean()/Ri,(alphasquared.mean()/risquared.mean()),(bsealphas.mean()/alphasquared.mean()),rsquared.mean())

In [154]:
#Factors
F1=FF5[0][['Mkt-RF']] #For portfolio beta only
F2=FF5[0][['Mkt-RF','SMB','HML']]
F3=FF5[0][['Mkt-RF','SMB','HML','RMW']]
F4=FF5[0][['Mkt-RF','SMB','HML','CMA']]
F5=FF5[0][['Mkt-RF','SMB','RMW','CMA']]
F6=FF5[0][['Mkt-RF','SMB','HML','RMW','CMA']]
#For Momentum Anomaly only
F71=FF6[['Mkt-RF','SMB','HML','Mom   ']]
F72=FF6[['Mkt-RF','SMB','HML','RMW','Mom   ']]
F73=FF6[['Mkt-RF','SMB','HML','CMA','Mom   ']]
F74=FF6[['Mkt-RF','SMB','RMW','CMA','Mom   ']]
F75=FF6[['Mkt-RF','SMB','HML','RMW','CMA','Mom   ']]

In [155]:
result=GRS(betaPortXs,F1)
#1-f.cdf(GRS(betaPortXs,F1)[0],GRS(betaPortXs,F1)[1],GRS(betaPortXs,F1)[2])

<ipython-input-153-b8f261998265>:27: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  alpha2=pd.np.array(alpha2)
<ipython-input-153-b8f261998265>:28: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  alphasquared=pd.np.array(alphasquared)
<ipython-input-153-b8f261998265>:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  returns=pd.np.array(returns)
<ipython-input-153-b8f261998265>:30: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  rsquared=pd.np.array(rsquared)
<ipython-input-153-b8f261998265>:31: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  bsealphas=pd.n

In [156]:
#Index name for GRS-Statistics
def indexname(f):
    name=""
    for data in list(f.columns):
        name=name+str(data)+" "
    return name
#Initialize
factorList=[F1,F2,F3,F4,F5,F6]
momfactorlist=[F2,F3,F4,F5,F6,F71,F72,F73,F74,F75]
portfolioList=[betaPortXs,niPortXs,varPortXs,rVarPortXs,acPortXs]
portnameList=["35 Size-NI Portffolio","25 Size-Var Portfolio","25 Size-RVar portfolios","25 Size-AC portfolios"] 
column_names=["Model Factors" , "GRS","p(GRS)" ,"A|ai|" ,"A|ai|/A|ri|","Aai^2/Ari^2","As^2(ai)/Aasi^2","A(R^2)"]
df=pd.DataFrame(columns=column_names)
counter=0

In [157]:
df.loc[0]=['25 Size Beta Portfolio',"","","","","","",""]
for fa in factorList:
    ans=GRS(betaPortXs,fa)
    counter+=1
    df.loc[counter]=[indexname(fa),ans[0],1-f.cdf(ans[0],ans[1],ans[2]),ans[3],ans[4],ans[5],ans[6],ans[7]]  

for port,name in list(zip(portfolioList[1:],portnameList)):
    counter+=1
    df.loc[counter]=[name,"","","","","","",""]
    for fa in factorList[1:]:
            ans=GRS(port,fa)
            counter+=1
            df.loc[counter]=[indexname(fa),ans[0],1-f.cdf(ans[0],ans[1],ans[2]),ans[3],ans[4],ans[5],ans[6],ans[7]]  
counter+=1       
df.loc[counter]=['25 Size Prior2-12 portfolios',"","","","","","",""]
for fa in momfactorlist:
    ans=GRS(momPortXs,fa)
    counter+=1
    df.loc[counter]=[indexname(fa),ans[0],1-f.cdf(ans[0],ans[1],ans[2]),ans[3],ans[4],ans[5],ans[6],ans[7]]  

<ipython-input-153-b8f261998265>:27: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  alpha2=pd.np.array(alpha2)
<ipython-input-153-b8f261998265>:28: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  alphasquared=pd.np.array(alphasquared)
<ipython-input-153-b8f261998265>:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  returns=pd.np.array(returns)
<ipython-input-153-b8f261998265>:30: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  rsquared=pd.np.array(rsquared)
<ipython-input-153-b8f261998265>:31: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  bsealphas=pd.n

In [158]:
df.set_index('Model Factors',drop=True,inplace=True)
#print (df.to_string(index=False))

In [159]:
df.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)

,GRS,p(GRS),A|ai|,A|ai|/A|ri|,Aai^2/Ari^2,As^2(ai)/Aasi^2,A(R^2)
Model Factors,,,,,,,
25 Size Beta Portfolio,,,,,,,
Mkt-RF,2.246,0.001,0.232,0.984,0.994,0.177,0.751
Mkt-RF SMB HML,1.584,0.036,0.107,0.453,0.201,0.378,0.886
Mkt-RF SMB HML RMW,1.801,0.01,0.081,0.342,0.127,0.582,0.892
Mkt-RF SMB HML CMA,1.473,0.066,0.093,0.392,0.165,0.467,0.887
Mkt-RF SMB RMW CMA,1.841,0.008,0.073,0.31,0.103,0.762,0.887
Mkt-RF SMB HML RMW CMA,1.853,0.007,0.073,0.311,0.103,0.726,0.894
35 Size-NI Portffolio,,,,,,,
Mkt-RF SMB HML,4.258,0,0.139,0.523,0.405,0.18,0.871


### 2.3 The five factor model vs FF three factor model

In [160]:
RMWexposure=GRS(FF5[0][['RMW']],F2)
CMAexposure=GRS(FF5[0][['CMA']],F2)


<ipython-input-153-b8f261998265>:27: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  alpha2=pd.np.array(alpha2)
<ipython-input-153-b8f261998265>:28: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  alphasquared=pd.np.array(alphasquared)
<ipython-input-153-b8f261998265>:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  returns=pd.np.array(returns)
<ipython-input-153-b8f261998265>:30: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  rsquared=pd.np.array(rsquared)
<ipython-input-153-b8f261998265>:31: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  bsealphas=pd.n

In [161]:
RMWexposure #return (grs,N,T-N-K,alpha2.mean(),alpha2.mean()/Ri,(alphasquared.mean()/risquared.mean()),(bsealphas.mean()/alphasquared.mean()),rsquared.mean())
CMAexposure

(12.127891912400196,
 1,
 614,
 0.19782252018339444,
 1.0643767845493455,
 1.1328979394876038,
 0.08246044901480115,
 0.5325692604844768)

In [162]:
modelCMA = sm.OLS(FF5[0][['CMA']], sm.add_constant(F2))
modelCMA.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    CMA   R-squared:                       0.535
Model:                            OLS   Adj. R-squared:                  0.533
Method:                 Least Squares   F-statistic:                     235.3
Date:                Fri, 18 Dec 2020   Prob (F-statistic):          1.30e-101
Time:                        17:16:19   Log-Likelihood:                -1074.2
No. Observations:                 618   AIC:                             2156.
Df Residuals:                     614   BIC:                             2174.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1978      0.057      3.482      0.001       0.086       0.309
Mkt-RF        -0.0976      0.013     -7.261      0.000      -0.124      -0.071
SMB           -0.0011      0.019     -0.059      0.953      -0.038       0.036
HML            0.4600      0.021     22.432      0.000       0.420       0.500
==============================================================================
Omnibus:                       21.640   Durbin-Watson:                   1.853
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               47.537
Skew:                           0.130   Prob(JB):                     4.76e-11
Kurtosis:                       4.334   Cond. No.                         4.83
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [163]:
modelRMW = sm.OLS(FF5[0][['RMW']], sm.add_constant(F2))
modelRMW.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    RMW   R-squared:                       0.139
Model:                            OLS   Adj. R-squared:                  0.135
Method:                 Least Squares   F-statistic:                     33.15
Date:                Fri, 18 Dec 2020   Prob (F-statistic):           7.24e-20
Time:                        17:16:19   Log-Likelihood:                -1326.2
No. Observations:                 618   AIC:                             2660.
Df Residuals:                     614   BIC:                             2678.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3573      0.085      4.184      0.000       0.190       0.525
Mkt-RF        -0.0688      0.020     -3.404      0.001      -0.108      -0.029
SMB           -0.2256      0.029     -7.904      0.000      -0.282      -0.170
HML            0.0113      0.031      0.366      0.714      -0.049       0.072
==============================================================================
Omnibus:                       96.067   Durbin-Watson:                   1.599
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1230.480
Skew:                          -0.073   Prob(JB):                    6.38e-268
Kurtosis:                       9.911   Cond. No.                         4.83
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### 2.4 An equivalent five factor model

In [164]:
#Equivalent five factore model
HMLexposure=GRS(FF5[0][['HML']],F5)
HMLexposure

<ipython-input-153-b8f261998265>:27: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  alpha2=pd.np.array(alpha2)
<ipython-input-153-b8f261998265>:28: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  alphasquared=pd.np.array(alphasquared)
<ipython-input-153-b8f261998265>:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  returns=pd.np.array(returns)
<ipython-input-153-b8f261998265>:30: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  rsquared=pd.np.array(rsquared)
<ipython-input-153-b8f261998265>:31: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  bsealphas=pd.n

(0.00030767938487675774,
 1,
 613,
 0.0014805408045509838,
 0.011397287209921635,
 0.00012989815574544327,
 3250.657736526995,
 0.5006616953708781)

In [165]:
model = sm.OLS(FF5[0][['HML']], sm.add_constant(F5))

In [166]:
model.fit().summary() #Constant is around 0.0016, does not really add value or means that nothing is lost in explanation of average returns 
#if we drop HML from the model

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    HML   R-squared:                       0.504
Model:                            OLS   Adj. R-squared:                  0.501
Method:                 Least Squares   F-statistic:                     155.7
Date:                Fri, 18 Dec 2020   Prob (F-statistic):           7.65e-92
Time:                        17:16:19   Log-Likelihood:                -1300.2
No. Observations:                 618   AIC:                             2610.
Df Residuals:                     613   BIC:                             2633.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0015      0.084     -0.018      0.986      -0.167       0.164
Mkt-RF         0.0171      0.021      0.830      0.407      -0.023       0.057
SMB            0.0268      0.029      0.932      0.352      -0.030       0.083
RMW            0.1514      0.039      3.868      0.000       0.075       0.228
CMA            1.0039      0.044     23.008      0.000       0.918       1.090
==============================================================================
Omnibus:                       42.454   Durbin-Watson:                   1.725
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              145.947
Skew:                          -0.206   Prob(JB):                     2.03e-32
Kurtosis:                       5.345   Cond. No.                         5.08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [167]:
##Calculate HMLO
HMLO=model.fit().params[0]+model.fit().resid
HMLO

Date
1963-07    0.243781
1963-08    1.945587
1963-09    0.093831
1963-10    1.723674
1963-11   -0.413395
             ...   
2014-08    0.134607
2014-09   -0.831468
2014-10   -1.701555
2014-11   -3.278496
2014-12    1.396545
Freq: M, Length: 618, dtype: float64

## 3. Market Beta

In [168]:
dt3=betaPortXs.mean()
dt3
dt3sd=betaPortXs.std()
dt3sd

SMALL LoBETA    4.404444
ME1 BETA2       5.092372
ME1 BETA3       5.946667
ME1 BETA4       6.514479
SMALL HiBETA    8.230605
ME2 BETA1       4.303275
ME2 BETA2       4.776218
ME2 BETA3       5.472472
ME2 BETA4       6.204383
ME2 BETA5       7.912692
ME3 BETA1       3.859098
ME3 BETA2       4.643507
ME3 BETA3       5.225233
ME3 BETA4       5.984497
ME3 BETA5       7.665540
ME4 BETA1       3.881337
ME4 BETA2       4.623011
ME4 BETA3       5.168256
ME4 BETA4       5.839759
ME4 BETA5       7.589291
BIG LoBETA      3.627947
ME5 BETA2       4.254481
ME5 BETA3       4.874660
ME5 BETA4       5.703102
BIG HiBETA      7.144848
dtype: float64

In [169]:

# List of tuples
students = [ (' ', '  ' , 'Mean' , ' ',' ',' ',' ',' ','SD',' ', ' '),
             (dt3[0], dt3[1], dt3[2] , dt3[3],dt3[4],' ',dt3sd[0], dt3sd[1], dt3sd[2] , dt3sd[3],dt3sd[4]) ,
             (dt3[5], dt3[6], dt3[7] , dt3[8],dt3[9],' ',dt3sd[5], dt3sd[6], dt3sd[7] , dt3sd[8],dt3sd[9]) ,
             (dt3[10], dt3[11], dt3[12] , dt3[13],dt3[14],' ',dt3sd[10], dt3sd[11], dt3sd[12] , dt3sd[13],dt3sd[14]) ,
             (dt3[15], dt3[16], dt3[17] , dt3[18],dt3[19],' ',dt3sd[15], dt3sd[16], dt3sd[17] , dt3sd[18],dt3sd[19]) ,
             (dt3[20], dt3[21], dt3[22] , dt3[23],dt3[24],' ',dt3sd[20], dt3sd[21], dt3sd[22] , dt3sd[23],dt3sd[24]) ]
# Create DataFrame object from a list of tuples
table3 = pd.DataFrame(students, columns = ['Low β' , '2', '3' , '4','High β',' ','Low β' , '2', '3' , '4','High β'], index=['','small', '2', '3' , '4' , 'Big'])
table3.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)

,Low β,2,3,4,High β,,Low β,2,3,4,High β
,,,Mean,,,,,,SD,,
small,0.729,0.891,0.928,0.972,0.797,,4.404,5.092,5.947,6.514,8.231
2,0.706,0.868,0.951,0.885,0.716,,4.303,4.776,5.472,6.204,7.913
3,0.689,0.863,0.839,0.805,0.753,,3.859,4.644,5.225,5.984,7.666
4,0.669,0.762,0.734,0.587,0.753,,3.881,4.623,5.168,5.84,7.589
Big,0.488,0.517,0.495,0.497,0.405,,3.628,4.254,4.875,5.703,7.145


In [170]:
#B/M=6/7 OP= 9 INV=10 PRIOR B= 8
dt3a=betaPort[6].mean()
dt3b=betaPort[9].mean()
# List of tuples
students = [ (' ', '  ' , 'B/M' , ' ',' ',' ',' ',' ','OP',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4]) ,
             (dt3a[5], dt3a[6], dt3a[7] , dt3a[8],dt3a[9],' ',dt3b[5], dt3b[6], dt3b[7] , dt3b[8],dt3b[9]) ,
             (dt3a[10], dt3a[11], dt3a[12] , dt3a[13],dt3a[14],' ',dt3b[10], dt3b[11], dt3b[12] , dt3b[13],dt3b[14]) ,
             (dt3a[15], dt3a[16], dt3a[17] , dt3a[18],dt3a[19],' ',dt3b[15],dt3b[16], dt3b[17] , dt3b[18],dt3b[19]) ,
             (dt3a[20], dt3a[21], dt3a[22] , dt3a[23],dt3a[24],' ',dt3b[20], dt3b[21], dt3b[22] , dt3b[23],dt3b[24]) ]
# Create DataFrame object from a list of tuples
table3a = pd.DataFrame(students, columns = ['Low β' , '2', '3' , '4','High β',' ','Low β' , '2', '3' , '4','High β'], index=['','small', '2', '3' , '4' , 'Big'])
table3a.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)

,Low β,2,3,4,High β,,Low β,2,3,4,High β
,,,B/M,,,,,,OP,,
small,0.978,1.028,1.032,1.032,0.935,,0.167,0.158,0.17,0.791,0.045
2,0.858,0.852,0.826,0.803,0.74,,0.266,0.26,0.377,0.341,0.241
3,0.82,0.773,0.729,0.695,0.662,,0.272,0.308,0.338,0.344,0.327
4,0.757,0.686,0.669,0.667,0.642,,0.315,0.382,0.321,0.343,0.323
Big,0.611,0.478,0.543,0.532,0.616,,0.404,0.467,0.372,0.367,0.332


In [171]:
#B/M=6/7 OP= 9 INV=10 PRIOR B= 8
dt3a=betaPort[10].mean()
dt3b=betaPort[8].mean()
# List of tuples
students = [ (' ', '  ' , 'Inv' , ' ',' ',' ',' ',' ','Prior-Beta',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4]) ,
             (dt3a[5], dt3a[6], dt3a[7] , dt3a[8],dt3a[9],' ',dt3b[5], dt3b[6], dt3b[7] , dt3b[8],dt3b[9]) ,
             (dt3a[10], dt3a[11], dt3a[12] , dt3a[13],dt3a[14],' ',dt3b[10], dt3b[11], dt3b[12] , dt3b[13],dt3b[14]) ,
             (dt3a[15], dt3a[16], dt3a[17] , dt3a[18],dt3a[19],' ',dt3b[15],dt3b[16], dt3b[17] , dt3b[18],dt3b[19]) ,
             (dt3a[20], dt3a[21], dt3a[22] , dt3a[23],dt3a[24],' ',dt3b[20], dt3b[21], dt3b[22] , dt3b[23],dt3b[24]) ]
# Create DataFrame object from a list of tuples
table3b = pd.DataFrame(students, columns = ['Low β' , '2', '3' , '4','High β',' ','Low β' , '2', '3' , '4','High β'], index=['','small', '2', '3' , '4' , 'Big'])
table3b.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)

,Low β,2,3,4,High β,,Low β,2,3,4,High β
,,,Inv,,,,,,Prior-Beta,,
small,0.098,0.087,0.096,0.095,0.123,,0.321,0.837,1.151,1.512,2.479
2,0.11,0.108,0.122,0.135,0.178,,0.381,0.839,1.154,1.507,2.397
3,0.111,0.115,0.124,0.142,0.195,,0.383,0.838,1.15,1.498,2.296
4,0.099,0.11,0.118,0.129,0.187,,0.397,0.84,1.147,1.497,2.233
Big,0.091,0.11,0.133,0.142,0.181,,0.423,0.828,1.144,1.48,2.097


In [172]:
##Continue to the deeper regression on each port
def regMarket(R,F):
    alpha=[]
    beta=[]
    tstata=[]
    tstatb=[]
    c=[]
    tstatc=[]
    d=[]
    tstatd=[]
    
    for p in R.columns:
        model = sm.OLS(R[:][p], sm.add_constant(F[:]))
        fit = model.fit()
        alpha.append(fit.params["const"])
        beta.append(fit.params[1])
        tstata.append(fit.tvalues[0])
        tstatb.append(fit.tvalues[1])
    
        
    return (alpha,beta,tstata,tstatb)
def regCAPM(R,F):
    alpha=[]
    beta=[]
    tstata=[]
    tstatb=[]
    c=[]
    tstatc=[]
    d=[]
    tstatd=[]
    
    for p in R.columns:
        model = sm.OLS(R[:][p], sm.add_constant(F[:]))
        fit = model.fit()
        alpha.append(fit.params["const"])
        beta.append(fit.params[1])
        tstata.append(fit.tvalues[0])
        tstatb.append(fit.tvalues[1])
        c.append(fit.params[2])
        tstatc.append(fit.tvalues[2])
        d.append(fit.params[3])
        tstatd.append(fit.tvalues[3])
        
    return (alpha,beta,tstata,tstatb,c,tstatc,d,tstatd)
def regFourFactor(R,F):
    alpha=[]
    beta=[]
    tstata=[]
    tstatb=[]
    c=[]
    tstatc=[]
    d=[]
    tstatd=[]
    e=[]
    tstate=[]
    for p in R.columns:
        model = sm.OLS(R[:][p], sm.add_constant(F[:]))
        fit = model.fit()
        alpha.append(fit.params["const"])
        beta.append(fit.params[1])
        tstata.append(fit.tvalues[0])
        tstatb.append(fit.tvalues[1])
        c.append(fit.params[2])
        tstatc.append(fit.tvalues[2])
        d.append(fit.params[3])
        tstatd.append(fit.params[3])
        e.append(fit.params[4])
        tstate.append(fit.tvalues[4])
    return (alpha,beta,tstata,tstatb,c,tstatc,d,tstatd,e,tstate)
            
def regFivefactor(R,F):
    alpha=[]
    beta=[]
    s=[]
    h=[]
    r=[]
    c=[]
    tstata=[]
    tstatb=[]
    tstats=[]
    tstath=[]
    tstatr=[]
    tstatc=[]
       
    for p in R.columns:
        model = sm.OLS(R[:][p], sm.add_constant(F[:]))
        fit = model.fit()
        alpha.append(fit.params["const"])
        beta.append(fit.params[1])
        s.append(fit.params[2])
        h.append(fit.params[3])
        r.append(fit.params[4])
        c.append(fit.params[5])
        tstata.append(fit.tvalues[0])
        tstatb.append(fit.tvalues[1])
        tstats.append(fit.tvalues[2])
        tstath.append(fit.tvalues[3])
        tstatr.append(fit.tvalues[4])
        tstatc.append(fit.tvalues[5])
    return (alpha,beta,tstata,tstatb,s,tstats,h,tstath,r,tstatr,c,tstatc)
def regSixfactor(R,F):
    alpha=[]
    beta=[]
    s=[]
    h=[]
    r=[]
    c=[]
    tstata=[]
    tstatb=[]
    tstats=[]
    tstath=[]
    tstatr=[]
    tstatc=[]
    m=[]
    tstatm=[]
       
    for p in R.columns:
        model = sm.OLS(R[:][p], sm.add_constant(F[:]))
        fit = model.fit()
        alpha.append(fit.params["const"])
        beta.append(fit.params[1])
        s.append(fit.params[2])
        h.append(fit.params[3])
        r.append(fit.params[4])
        c.append(fit.params[5])
        m.append(fit.params[6])
        tstata.append(fit.tvalues[0])
        tstatb.append(fit.tvalues[1])
        tstats.append(fit.tvalues[2])
        tstath.append(fit.tvalues[3])
        tstatr.append(fit.tvalues[4])
        tstatc.append(fit.tvalues[5])
        tstatm.append(fit.tvalues[6])
    return (alpha,beta,tstata,tstatb,s,tstats,h,tstath,r,tstatr,c,tstatc,m,tstatm)

In [173]:

result3a=regMarket(betaPortXs,F1)


In [174]:

# List of tuples
regcapmdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ('', '  ' , 'b' , ' ',' ',' ',' ',' ','t(b)',' ', ' '),
             (result3a[1][0], result3a[1][1], result3a[1][2] , result3a[1][3],result3a[1][4],' ',result3a[3][0], result3a[3][1], result3a[3][2] , result3a[3][3],result3a[3][4]) ,
             (result3a[1][5], result3a[1][6], result3a[1][7] , result3a[1][8],result3a[1][9],' ',result3a[3][5], result3a[3][6], result3a[3][7] , result3a[3][8],result3a[3][9]) ,
             (result3a[1][10], result3a[1][11], result3a[1][12] , result3a[1][13],result3a[1][14],' ',result3a[3][10], result3a[3][11], result3a[3][12] , result3a[3][13],result3a[3][14]) ,
             (result3a[1][15], result3a[1][16], result3a[1][17] , result3a[1][18],result3a[1][19],' ',result3a[3][15], result3a[3][16], result3a[3][17] , result3a[3][18],result3a[3][19]) ,
             (result3a[1][20], result3a[1][21], result3a[1][22] , result3a[1][23],result3a[1][24],' ',result3a[3][20], result3a[3][21], result3a[3][22] , result3a[3][23],result3a[3][24])]
# Create DataFrame object from a list of tuples
table4a = pd.DataFrame(regcapmdata, columns = ['Low β' , '2', '3' , '4','High β',' ','Low β' , '2', '3' , '4','High β'], index=['','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big'])
table4a.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)

,Low β,2,3,4,High β,,Low β,2,3,4,High β
,,,a,,,,,,t(a),,
small,0.347,0.434,0.399,0.359,0.046,,3,3.41,2.654,2.412,0.232
2,0.316,0.411,0.41,0.271,-0.071,,2.991,3.916,3.717,2.177,-0.45
3,0.326,0.389,0.304,0.195,-0.027,,3.698,4.645,3.268,1.801,-0.191
4,0.304,0.278,0.186,-0.029,-0.026,,3.434,3.755,2.443,-0.33,-0.192
Big,0.15,0.067,-0.03,-0.113,-0.314,,1.777,1.047,-0.48,-1.424,-2.32
,,,b,,,,,,t(b),,
small,0.752,0.899,1.044,1.207,1.479,,29.114,31.622,31.145,36.3,33.183
2,0.768,0.901,1.065,1.209,1.549,,32.573,38.49,43.256,43.509,44.289
3,0.715,0.933,1.053,1.202,1.537,,36.324,49.925,50.73,49.795,49.36


In [175]:
## Five Factor regression with HML->HMLO
test=F2.copy()
test=test.drop(['HML'],axis=1)
test['HMLO']=HMLO
test['RMW']=F5['RMW']
test['CMA']=F5['CMA']

In [176]:
result3a=regFivefactor(betaPortXs,test) #For five factor regression

In [177]:
#List of tuples
reg5Fdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ('', '  ' , 'b' , ' ',' ',' ',' ',' ','t(b)',' ', ' '),
             (result3a[1][0], result3a[1][1], result3a[1][2] , result3a[1][3],result3a[1][4],' ',result3a[3][0], result3a[3][1], result3a[3][2] , result3a[3][3],result3a[3][4]) ,
             (result3a[1][5], result3a[1][6], result3a[1][7] , result3a[1][8],result3a[1][9],' ',result3a[3][5], result3a[3][6], result3a[3][7] , result3a[3][8],result3a[3][9]) ,
             (result3a[1][10], result3a[1][11], result3a[1][12] , result3a[1][13],result3a[1][14],' ',result3a[3][10], result3a[3][11], result3a[3][12] , result3a[3][13],result3a[3][14]) ,
             (result3a[1][15], result3a[1][16], result3a[1][17] , result3a[1][18],result3a[1][19],' ',result3a[3][15], result3a[3][16], result3a[3][17] , result3a[3][18],result3a[3][19]) ,
             (result3a[1][20], result3a[1][21], result3a[1][22] , result3a[1][23],result3a[1][24],' ',result3a[3][20], result3a[3][21], result3a[3][22] , result3a[3][23],result3a[3][24]),
             ('', '  ' , 's' , ' ',' ',' ',' ',' ','t(s)',' ', ' '),
             (result3a[4][0], result3a[4][1], result3a[4][2] , result3a[4][3],result3a[4][4],' ',result3a[5][0], result3a[5][1], result3a[5][2] , result3a[5][3],result3a[5][4]),
             (result3a[4][5], result3a[4][6], result3a[4][7] , result3a[4][8],result3a[4][9],' ',result3a[5][5], result3a[5][6], result3a[5][7] , result3a[5][8],result3a[5][9]),
             (result3a[4][10], result3a[4][11], result3a[4][12] , result3a[4][13],result3a[4][14],' ',result3a[5][10], result3a[5][11], result3a[5][12] , result3a[5][13],result3a[5][14]),
             (result3a[4][15], result3a[4][16], result3a[4][17] , result3a[4][18],result3a[4][19],' ',result3a[5][15], result3a[5][16], result3a[5][17] , result3a[5][18],result3a[5][19]),
             (result3a[4][20], result3a[4][21], result3a[4][22] , result3a[4][23],result3a[4][24],' ',result3a[5][20], result3a[5][21], result3a[5][22] , result3a[5][23],result3a[5][24]),
             ('', '  ' , 'h' , ' ',' ',' ',' ',' ','t(h)',' ', ' '),
             (result3a[6][0], result3a[6][1], result3a[6][2] , result3a[6][3],result3a[6][4],' ',result3a[7][0], result3a[7][1], result3a[7][2] , result3a[7][3],result3a[7][4]) ,
             (result3a[6][5], result3a[6][6], result3a[6][7] , result3a[6][8],result3a[6][9],' ',result3a[7][5], result3a[7][6], result3a[7][7] , result3a[7][8],result3a[7][9]) ,
             (result3a[6][10], result3a[6][11], result3a[6][12] , result3a[6][13],result3a[6][14],' ',result3a[7][10], result3a[7][11], result3a[7][12] , result3a[7][13],result3a[7][14]) ,
             (result3a[6][15], result3a[6][16], result3a[6][17] , result3a[6][18],result3a[6][19],' ',result3a[7][15], result3a[7][16], result3a[7][17] , result3a[7][18],result3a[7][19]) ,
             (result3a[6][20], result3a[6][21], result3a[6][22] , result3a[6][23],result3a[6][24],' ',result3a[7][20], result3a[7][21], result3a[7][22] , result3a[7][23],result3a[7][24]), 
             ('', '  ' , 'r' , ' ',' ',' ',' ',' ','t(r)',' ', ' '),
             (result3a[8][0], result3a[8][1], result3a[8][2] , result3a[8][3],result3a[8][4],' ',result3a[9][0], result3a[9][1], result3a[9][2] , result3a[9][3],result3a[9][4]) ,
             (result3a[8][5], result3a[8][6], result3a[8][7] , result3a[8][8],result3a[8][9],' ',result3a[9][5], result3a[9][6], result3a[9][7] , result3a[9][8],result3a[9][9]) ,
             (result3a[8][10], result3a[8][11], result3a[8][12] , result3a[8][13],result3a[8][14],' ',result3a[9][10], result3a[9][11], result3a[9][12] , result3a[9][13],result3a[9][14]) ,
             (result3a[8][15], result3a[8][16], result3a[8][17] , result3a[8][18],result3a[8][19],' ',result3a[9][15], result3a[9][16], result3a[9][17] , result3a[9][18],result3a[9][19]) ,
             (result3a[8][20], result3a[8][21], result3a[8][22] , result3a[8][23],result3a[8][24],' ',result3a[9][20], result3a[9][21], result3a[9][22] , result3a[9][23],result3a[9][24]),
             ('', '  ' , 'c' , ' ',' ',' ',' ',' ','t(c)',' ', ' '),
             (result3a[10][0], result3a[10][1], result3a[10][2] , result3a[10][3],result3a[10][4],' ',result3a[11][0], result3a[11][1], result3a[11][2] , result3a[11][3],result3a[11][4]) ,
             (result3a[10][5], result3a[10][6], result3a[10][7] , result3a[10][8],result3a[10][9],' ',result3a[11][5], result3a[11][6], result3a[11][7] , result3a[11][8],result3a[11][9]) ,
             (result3a[10][10], result3a[10][11], result3a[10][12] , result3a[10][13],result3a[10][14],' ',result3a[11][10], result3a[11][11], result3a[11][12] , result3a[11][13],result3a[11][14]) ,
             (result3a[10][15], result3a[10][16], result3a[10][17] , result3a[10][18],result3a[10][19],' ',result3a[11][15], result3a[11][16], result3a[11][17] , result3a[11][18],result3a[11][19]) ,
             (result3a[10][20], result3a[10][21], result3a[10][22] , result3a[10][23],result3a[10][24],' ',result3a[11][20], result3a[11][21], result3a[11][22] , result3a[11][23],result3a[11][24])]



# Create DataFrame object from a list of tuples
table4b = pd.DataFrame(reg5Fdata, columns = ['Low β' , '2', '3' , '4','High β',' ','Low β' , '2', '3' , '4','High β'], index=['','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big'])
table4b.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)

,Low β,2,3,4,High β,,Low β,2,3,4,High β
,,,a,,,,,,t(a),,
small,0.081,0.074,0.062,0.079,0.002,,1.227,1.154,0.936,1.295,0.026
2,0.03,0.034,0.061,-0.061,-0.142,,0.4,0.615,1.04,-1.025,-1.88
3,0.069,0.095,-0.004,-0.104,-0.023,,0.913,1.603,-0.057,-1.477,-0.261
4,0.045,-0.025,-0.094,-0.247,0.098,,0.546,-0.395,-1.448,-3.14,0.881
Big,-0.083,-0.085,-0.089,-0.093,-0.055,,-1.256,-1.611,-1.508,-1.147,-0.407
,,,b,,,,,,t(b),,
small,0.663,0.81,0.896,1.03,1.144,,41.296,51.585,55.817,69.578,56.342
2,0.73,0.861,0.997,1.098,1.295,,40.537,64.261,70.15,75.938,70.331
3,0.75,0.922,1.019,1.132,1.312,,40.792,64.138,67.786,65.975,59.974


### 4.NI Net Share Issues

In [178]:
dt3=niPortXs.mean()
dt3
dt3sd=niPortXs.std()
dt3sd.head()

SMALL NegNI     5.667709
SMALL ZeroNI    5.705591
SMALL LoNI      6.362067
ME1 NI2         6.432492
ME1 NI3         6.857032
dtype: float64

In [179]:
# List of tuples
students = [ (' ',' ',' ','Mean',' ',' ',' ',' ',' ',' ',' ','SD',' ',' ', ' '),
             (dt3[0], dt3[1], dt3[2] , dt3[3],dt3[4],dt3[5], dt3[6],' ',dt3sd[0], dt3sd[1], dt3sd[2] , dt3sd[3],dt3sd[4],dt3sd[5], dt3sd[6]),
             (dt3[7], dt3[8], dt3[9] , dt3[10],dt3[11],dt3[12], dt3[13],' ',dt3sd[7], dt3sd[8], dt3sd[9] , dt3sd[10],dt3sd[11],dt3sd[12], dt3sd[13]),
             (dt3[14], dt3[15], dt3[16],dt3[17],dt3[18],dt3[19], dt3[20],' ',dt3sd[14], dt3sd[15], dt3sd[16] , dt3sd[17],dt3sd[18],dt3sd[19], dt3sd[20]),
             (dt3[21], dt3[22], dt3[23] , dt3[24],dt3[25],dt3[26], dt3[27],' ',dt3sd[21], dt3sd[22], dt3sd[23] , dt3sd[24],dt3sd[25],dt3sd[26], dt3sd[27]),
             (dt3[28], dt3[29], dt3[30] , dt3[31],dt3[32],dt3[33], dt3[34],' ',dt3sd[28], dt3sd[29], dt3sd[30] , dt3sd[31],dt3sd[32],dt3sd[33], dt3sd[34])]
# Create DataFrame object from a list of tuples
table3 = pd.DataFrame(students, columns = ['Neg','Zero','Low' , '2', '3' , '4','High ',' ','Neg','Zero','Low', '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table3.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)

,Neg,Zero,Low,2,3,4,High,,Neg,Zero,Low,2,3,4,High
,,,,Mean,,,,,,,,SD,,,
small,1.041,0.763,0.9,0.95,1.021,0.698,0.243,,5.668,5.706,6.362,6.432,6.857,7.121,7.738
2,0.91,0.803,0.914,0.913,0.851,0.813,0.306,,5.304,5.633,5.839,6.127,6.32,6.53,7.08
3,0.964,0.752,0.834,0.93,0.833,0.723,0.287,,5.078,5.431,5.282,5.628,5.835,6.017,6.548
4,0.941,0.56,0.653,0.687,0.821,0.622,0.363,,4.912,4.885,5.07,5.336,5.455,5.809,6.186
Big,0.616,0.609,0.491,0.479,0.59,0.432,0.177,,4.176,4.787,4.248,4.551,4.859,5.418,5.121


In [180]:
#6: B/M  10:OP 11:inv  8:NI
dt3a=niPort[6].mean()
dt3b=niPort[10].mean()
dt3b['BIG ZeroNI']=niPort[10]['BIG ZeroNI'].median()#Some error in the dataset. Already screenshot
dt3b.round(6)
# List of tuples
students = [ (' ',' ',' ','B/M',' ',' ',' ',' ',' ',' ',' ','OP',' ',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],dt3a[5], dt3a[6],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4],dt3b[5], dt3b[6]),
             (dt3a[7], dt3a[8], dt3a[9] , dt3a[10],dt3a[11],dt3a[12], dt3a[13],' ',dt3b[7], dt3b[8], dt3b[9] , dt3b[10],dt3b[11],dt3b[12], dt3b[13]),
             (dt3a[14], dt3a[15], dt3a[16],dt3a[17],dt3a[18],dt3a[19], dt3a[20],' ',dt3b[14], dt3b[15], dt3b[16] , dt3b[17],dt3b[18],dt3b[19], dt3b[20]),
             (dt3a[21], dt3a[22], dt3a[23] , dt3a[24],dt3a[25],dt3a[26], dt3a[27],' ',dt3b[21], dt3b[22], dt3b[23] , dt3b[24],dt3b[25],dt3b[26], dt3b[27]),
             (dt3a[28], dt3a[29], dt3a[30] , dt3a[31],dt3a[32],dt3a[33], dt3a[34],' ',dt3b[28], dt3b[29], dt3b[30] , dt3b[31],dt3b[32],dt3b[33], dt3b[34])]
# Create DataFrame object from a list of tuples
table3a = pd.DataFrame(students, columns = ['Neg','Zero','Low' , '2', '3' , '4','High ',' ','Neg','Zero','Low', '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table3a.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)

,Neg,Zero,Low,2,3,4,High,,Neg,Zero,Low,2,3,4,High
,,,,B/M,,,,,,,,OP,,,
small,1.045,1.153,1.005,0.916,0.853,0.749,0.682,,0.605,0.237,0.213,0.228,0.212,0.097,-0.091
2,0.845,0.981,0.838,0.771,0.71,0.661,0.619,,0.308,0.43,0.275,0.281,0.359,0.251,0.25
3,0.764,0.882,0.809,0.701,0.657,0.633,0.625,,0.332,0.316,0.301,0.301,0.32,0.32,0.281
4,0.674,0.858,0.755,0.65,0.641,0.642,0.652,,0.393,0.29,0.3,0.303,0.376,0.305,0.249
Big,0.569,0.731,0.551,0.487,0.494,0.58,0.674,,0.427,0.293,0.346,0.369,0.487,0.333,0.302


In [181]:
#6: B/M  10:OP 11:inv  8:NI
dt3a=niPort[11].mean()
dt3b=niPort[8].mean()
# List of tuples
students = [ (' ',' ',' ','Inv',' ',' ',' ',' ',' ',' ',' ','NI',' ',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],dt3a[5], dt3a[6],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4],dt3b[5], dt3b[6]),
             (dt3a[7], dt3a[8], dt3a[9] , dt3a[10],dt3a[11],dt3a[12], dt3a[13],' ',dt3b[7], dt3b[8], dt3b[9] , dt3b[10],dt3b[11],dt3b[12], dt3b[13]),
             (dt3a[14], dt3a[15], dt3a[16],dt3a[17],dt3a[18],dt3a[19], dt3a[20],' ',dt3b[14], dt3b[15], dt3b[16] , dt3b[17],dt3b[18],dt3b[19], dt3b[20]),
             (dt3a[21], dt3a[22], dt3a[23] , dt3a[24],dt3a[25],dt3a[26], dt3a[27],' ',dt3b[21], dt3b[22], dt3b[23] , dt3b[24],dt3b[25],dt3b[26], dt3b[27]),
             (dt3a[28], dt3a[29], dt3a[30] , dt3a[31],dt3a[32],dt3a[33], dt3a[34],' ',dt3b[28], dt3b[29], dt3b[30] , dt3b[31],dt3b[32],dt3b[33], dt3b[34])]
# Create DataFrame object from a list of tuples
table3b = pd.DataFrame(students, columns = ['Neg','Zero','Low' , '2', '3' , '4','High ',' ','Neg','Zero','Low', '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table3b.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)

,Neg,Zero,Low,2,3,4,High,,Neg,Zero,Low,2,3,4,High
,,,,Inv,,,,,,,,NI,,,
small,0.054,0.057,0.068,0.073,0.088,0.131,0.43,,-5.044,0,0.137,0.522,1.284,3.837,35.62
2,0.072,0.094,0.092,0.101,0.124,0.16,0.425,,-4.51,0,0.14,0.518,1.287,3.899,33.8
3,0.071,0.097,0.087,0.105,0.13,0.175,0.392,,-3.858,0,0.135,0.519,1.289,3.953,28.675
4,0.069,0.071,0.081,0.099,0.118,0.165,0.337,,-3.38,0,0.139,0.515,1.281,3.949,24.371
Big,0.078,0.091,0.088,0.111,0.126,0.163,0.317,,-1.986,0,0.127,0.508,1.276,3.838,22.406


### Regression

In [182]:
result3a=regCAPM(niPortXs,F2)
dt3a=result3a[0]
dt3b=result3a[2]

In [183]:
# List of tuples
regcapmdata = [ (' ',' ',' ','a',' ',' ',' ',' ',' ',' ',' ','t(a)',' ',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],dt3a[5], dt3a[6],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4],dt3b[5], dt3b[6]),
             (dt3a[7], dt3a[8], dt3a[9] , dt3a[10],dt3a[11],dt3a[12], dt3a[13],' ',dt3b[7], dt3b[8], dt3b[9] , dt3b[10],dt3b[11],dt3b[12], dt3b[13]),
             (dt3a[14], dt3a[15], dt3a[16],dt3a[17],dt3a[18],dt3a[19], dt3a[20],' ',dt3b[14], dt3b[15], dt3b[16] , dt3b[17],dt3b[18],dt3b[19], dt3b[20]),
             (dt3a[21], dt3a[22], dt3a[23] , dt3a[24],dt3a[25],dt3a[26], dt3a[27],' ',dt3b[21], dt3b[22], dt3b[23] , dt3b[24],dt3b[25],dt3b[26], dt3b[27]),
             (dt3a[28], dt3a[29], dt3a[30] , dt3a[31],dt3a[32],dt3a[33], dt3a[34],' ',dt3b[28], dt3b[29], dt3b[30] , dt3b[31],dt3b[32],dt3b[33], dt3b[34])]
# Create DataFrame object from a list of tuples
table4a = pd.DataFrame(regcapmdata, columns = ['Neg','Zero','Low' , '2', '3' , '4','High ',' ','Neg','Zero','Low', '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table4a.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)

,Neg,Zero,Low,2,3,4,High,,Neg,Zero,Low,2,3,4,High
,,,,a,,,,,,,,t(a),,,
small,0.198,-0.025,0.007,0.075,0.18,-0.148,-0.568,,3.356,-0.314,0.1,1.089,2.449,-1.951,-6.001
2,0.109,0.043,0.083,0.097,0.055,0.044,-0.435,,1.937,0.381,1.195,1.367,0.825,0.666,-5.703
3,0.238,0.035,0.049,0.184,0.128,0.015,-0.367,,3.581,0.294,0.669,2.612,1.787,0.204,-4.496
4,0.239,-0.014,-0.048,0.011,0.215,0.011,-0.279,,3.493,-0.11,-0.629,0.141,2.935,0.128,-2.953
Big,0.142,0.162,0.068,0.061,0.145,0.02,-0.363,,2.63,1.159,0.986,0.955,1.998,0.234,-4.27


In [184]:
result3a=regFivefactor(niPortXs,test)  #test already include HMLO
dt3a=result3a[0] 
dt3b=result3a[2]
dt3c=result3a[6] 

dt3d=result3a[7] 
dt3e=result3a[8]
dt3f=result3a[9]
dt3g=result3a[10]
dt3h=result3a[11]

In [185]:
reg5FdataNI = [ (' ',' ',' ','a',' ',' ',' ',' ',' ',' ',' ','t(a)',' ',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],dt3a[5], dt3a[6],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4],dt3b[5], dt3b[6]),
             (dt3a[7], dt3a[8], dt3a[9] , dt3a[10],dt3a[11],dt3a[12], dt3a[13],' ',dt3b[7], dt3b[8], dt3b[9] , dt3b[10],dt3b[11],dt3b[12], dt3b[13]),
             (dt3a[14], dt3a[15], dt3a[16],dt3a[17],dt3a[18],dt3a[19], dt3a[20],' ',dt3b[14], dt3b[15], dt3b[16] , dt3b[17],dt3b[18],dt3b[19], dt3b[20]),
             (dt3a[21], dt3a[22], dt3a[23] , dt3a[24],dt3a[25],dt3a[26], dt3a[27],' ',dt3b[21], dt3b[22], dt3b[23] , dt3b[24],dt3b[25],dt3b[26], dt3b[27]),
             (dt3a[28], dt3a[29], dt3a[30] , dt3a[31],dt3a[32],dt3a[33], dt3a[34],' ',dt3b[28], dt3b[29], dt3b[30] , dt3b[31],dt3b[32],dt3b[33], dt3b[34]),
             (' ',' ',' ','h',' ',' ',' ',' ',' ',' ',' ','t(h)',' ',' ', ' '),
             (dt3c[0], dt3c[1], dt3c[2] , dt3c[3],dt3c[4],dt3c[5], dt3c[6],' ',dt3d[0], dt3d[1], dt3d[2] , dt3d[3],dt3d[4],dt3d[5], dt3d[6]),
             (dt3c[7], dt3c[8], dt3c[9] , dt3c[10],dt3c[11],dt3c[12], dt3c[13],' ',dt3d[7], dt3d[8], dt3d[9] , dt3d[10],dt3d[11],dt3d[12], dt3d[13]),
             (dt3c[14], dt3c[15], dt3c[16],dt3c[17],dt3c[18],dt3c[19], dt3c[20],' ',dt3d[14], dt3d[15], dt3d[16] , dt3d[17],dt3d[18],dt3d[19], dt3d[20]),
             (dt3c[21], dt3c[22], dt3c[23] , dt3c[24],dt3c[25],dt3c[26], dt3c[27],' ',dt3d[21], dt3d[22], dt3d[23] , dt3d[24],dt3d[25],dt3d[26], dt3d[27]),
             (dt3c[28], dt3c[29], dt3c[30] , dt3c[31],dt3c[32],dt3c[33], dt3c[34],' ',dt3d[28], dt3d[29], dt3d[30] , dt3d[31],dt3d[32],dt3d[33], dt3d[34]), 
             (' ',' ',' ','r',' ',' ',' ',' ',' ',' ',' ','t(r)',' ',' ', ' '),
             (dt3e[0], dt3e[1], dt3e[2] , dt3e[3],dt3e[4],dt3e[5], dt3e[6],' ',dt3f[0], dt3f[1], dt3f[2] , dt3f[3],dt3f[4],dt3f[5], dt3f[6]),
             (dt3e[7], dt3e[8], dt3e[9] , dt3e[10],dt3e[11],dt3e[12], dt3e[13],' ',dt3f[7], dt3f[8], dt3f[9] , dt3f[10],dt3f[11],dt3f[12], dt3f[13]),
             (dt3e[14], dt3e[15], dt3e[16],dt3e[17],dt3e[18],dt3e[19], dt3e[20],' ',dt3f[14], dt3f[15], dt3f[16] , dt3f[17],dt3f[18],dt3f[19], dt3f[20]),
             (dt3e[21], dt3e[22], dt3e[23] , dt3e[24],dt3e[25],dt3e[26], dt3e[27],' ',dt3f[21], dt3f[22], dt3f[23] , dt3f[24],dt3f[25],dt3f[26], dt3f[27]),
             (dt3e[28], dt3e[29], dt3e[30] , dt3e[31],dt3e[32],dt3e[33], dt3e[34],' ',dt3f[28], dt3f[29], dt3f[30] , dt3f[31],dt3f[32],dt3f[33], dt3f[34]),
             (' ',' ',' ','c',' ',' ',' ',' ',' ',' ',' ','t(c)',' ',' ', ' '),
             (dt3g[0], dt3g[1], dt3g[2] , dt3g[3],dt3g[4],dt3g[5], dt3g[6],' ',dt3h[0], dt3h[1], dt3h[2] , dt3h[3],dt3h[4],dt3h[5], dt3h[6]),
             (dt3g[7], dt3g[8], dt3g[9] , dt3g[10],dt3g[11],dt3g[12], dt3g[13],' ',dt3h[7], dt3h[8], dt3h[9] , dt3h[10],dt3h[11],dt3h[12], dt3h[13]),
             (dt3g[14], dt3g[15], dt3g[16],dt3g[17],dt3g[18],dt3g[19], dt3g[20],' ',dt3h[14], dt3h[15], dt3h[16] , dt3h[17],dt3h[18],dt3h[19], dt3h[20]),
             (dt3g[21], dt3g[22], dt3g[23] , dt3g[24],dt3g[25],dt3g[26], dt3g[27],' ',dt3h[21], dt3h[22], dt3h[23] , dt3h[24],dt3h[25],dt3h[26], dt3h[27]),
             (dt3g[28], dt3g[29], dt3g[30] , dt3g[31],dt3g[32],dt3g[33], dt3g[34],' ',dt3h[28], dt3h[29], dt3h[30] , dt3h[31],dt3h[32],dt3h[33], dt3h[34])]


# Create DataFrame object from a list of tuples
table4b = pd.DataFrame(reg5FdataNI, columns =['Neg','Zero','Low' , '2', '3' , '4','High ',' ','Neg','Zero','Low', '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big'])
table4b.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)

,Neg,Zero,Low,2,3,4,High,,Neg,Zero,Low,2,3,4,High
,,,,a,,,,,,,,t(a),,,
small,0.103,-0.065,-0.015,0.054,0.246,-0.033,-0.347,,1.754,-0.79,-0.202,0.77,3.4,-0.448,-4.103
2,-0.033,-0.055,0.001,0.009,0.043,0.14,-0.241,,-0.631,-0.479,0.018,0.131,0.627,2.094,-3.405
3,0.069,-0.03,-0.051,0.124,0.123,0.093,-0.121,,1.152,-0.242,-0.691,1.72,1.662,1.251,-1.651
4,0.066,-0.215,-0.199,-0.103,0.225,0.156,0.005,,1.033,-1.713,-2.68,-1.368,2.979,1.95,0.058
Big,0.011,-0.014,-0.08,0.037,0.157,0.197,-0.197,,0.215,-0.103,-1.211,0.566,2.106,2.377,-2.399
,,,,h,,,,,,,,t(h),,,
small,0.264,0.207,0.218,0.128,-0.029,-0.011,-0.203,,9.422,5.227,6.14,3.806,-0.835,-0.308,-5.021
2,0.242,0.306,0.254,0.044,0.036,-0.013,-0.127,,9.677,5.608,7.654,1.295,1.081,-0.414,-3.732
3,0.197,0.226,0.335,0.105,0.004,0.057,-0.052,,6.865,3.837,9.531,3.059,0.122,1.587,-1.475


## 5.Volatility (r-Var portfolios)

In [186]:
dt3=rVarPortXs.mean()
dt3
dt3sd=rVarPortXs.std()
dt3sd
# List of tuples
students = [ (' ', '  ' , 'Mean' , ' ',' ',' ',' ',' ','SD',' ', ' '),
             (dt3[0], dt3[1], dt3[2] , dt3[3],dt3[4],' ',dt3sd[0], dt3sd[1], dt3sd[2] , dt3sd[3],dt3sd[4]) ,
             (dt3[5], dt3[6], dt3[7] , dt3[8],dt3[9],' ',dt3sd[5], dt3sd[6], dt3sd[7] , dt3sd[8],dt3sd[9]) ,
             (dt3[10], dt3[11], dt3[12] , dt3[13],dt3[14],' ',dt3sd[10], dt3sd[11], dt3sd[12] , dt3sd[13],dt3sd[14]) ,
             (dt3[15], dt3[16], dt3[17] , dt3[18],dt3[19],' ',dt3sd[15], dt3sd[16], dt3sd[17] , dt3sd[18],dt3sd[19]) ,
             (dt3[20], dt3[21], dt3[22] , dt3[23],dt3[24],' ',dt3sd[20], dt3sd[21], dt3sd[22] , dt3sd[23],dt3sd[24]) ]
# Create DataFrame object from a list of tuples
table3 = pd.DataFrame(students, columns = ['Low ' , '2', '3' , '4','High ',' ','Low ' , '2', '3' , '4','High '], index=['','small', '2', '3' , '4' , 'Big'])
table3.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)#Pnael a )

,Low,2,3,4,High,,Low,2,3,4,High
,,,Mean,,,,,,SD,,
small,1.005,1.192,1.062,0.826,-0.201,,4.192,5.638,6.486,7.527,8.997
2,0.912,1.035,1.069,0.959,0.183,,4.121,5.295,5.963,6.819,8.425
3,0.746,0.914,0.929,0.928,0.346,,3.799,4.854,5.423,6.207,7.781
4,0.716,0.759,0.748,0.775,0.441,,3.837,4.527,5.186,5.738,7.408
Big,0.468,0.509,0.505,0.483,0.466,,3.655,4.153,4.564,5.026,6.422


In [187]:
#B/M=6/7 OP= 9 INV=10 PRIOR B= 8
dt3a=rVarPort[6].mean()
dt3b=rVarPort[9].mean()
# List of tuples
students = [ (' ', '  ' , 'B/M' , ' ',' ',' ',' ',' ','OP',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4]) ,
             (dt3a[5], dt3a[6], dt3a[7] , dt3a[8],dt3a[9],' ',dt3b[5], dt3b[6], dt3b[7] , dt3b[8],dt3b[9]) ,
             (dt3a[10], dt3a[11], dt3a[12] , dt3a[13],dt3a[14],' ',dt3b[10], dt3b[11], dt3b[12] , dt3b[13],dt3b[14]) ,
             (dt3a[15], dt3a[16], dt3a[17] , dt3a[18],dt3a[19],' ',dt3b[15],dt3b[16], dt3b[17] , dt3b[18],dt3b[19]) ,
             (dt3a[20], dt3a[21], dt3a[22] , dt3a[23],dt3a[24],' ',dt3b[20], dt3b[21], dt3b[22] , dt3b[23],dt3b[24]) ]
# Create DataFrame object from a list of tuples
table3a = pd.DataFrame(students, columns = ['Low ' , '2', '3' , '4','High ',' ','Low ' , '2', '3' , '4','High '], index=['','small', '2', '3' , '4' , 'Big'])
table3a.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)


,Low,2,3,4,High,,Low,2,3,4,High
,,,B/M,,,,,,OP,,
small,0.99,0.94,0.908,0.884,0.924,,0.243,0.317,0.36,0.494,0.001
2,0.861,0.811,0.778,0.75,0.713,,0.329,0.338,0.318,0.3,0.204
3,0.809,0.74,0.706,0.686,0.657,,0.301,0.328,0.338,0.347,0.292
4,0.77,0.699,0.663,0.648,0.621,,0.311,0.369,0.39,0.336,0.363
Big,0.566,0.552,0.551,0.556,0.53,,0.379,0.392,0.438,0.396,0.428


In [188]:
#B/M=6/7 OP= 9 INV=10 PRIOR B= 8
dt3a=rVarPort[10].mean()
dt3b=rVarPort[8].mean()
# List of tuples
students = [ (' ', '  ' , 'Inv' , ' ',' ',' ',' ',' ','RVar',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4]) ,
             (dt3a[5], dt3a[6], dt3a[7] , dt3a[8],dt3a[9],' ',dt3b[5], dt3b[6], dt3b[7] , dt3b[8],dt3b[9]) ,
             (dt3a[10], dt3a[11], dt3a[12] , dt3a[13],dt3a[14],' ',dt3b[10], dt3b[11], dt3b[12] , dt3b[13],dt3b[14]) ,
             (dt3a[15], dt3a[16], dt3a[17] , dt3a[18],dt3a[19],' ',dt3b[15],dt3b[16], dt3b[17] , dt3b[18],dt3b[19]) ,
             (dt3a[20], dt3a[21], dt3a[22] , dt3a[23],dt3a[24],' ',dt3b[20], dt3b[21], dt3b[22] , dt3b[23],dt3b[24]) ]
# Create DataFrame object from a list of tuples
table3b = pd.DataFrame(students, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table3b.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)

,Low,2,3,4,High,,Low,2,3,4,High
,,,Inv,,,,,,RVar,,
small,0.103,0.129,0.159,0.187,0.191,,2.044,4.646,7.539,12.941,41.17
2,0.096,0.122,0.15,0.188,0.253,,1.411,2.899,4.399,6.664,16.815
3,0.096,0.114,0.134,0.166,0.25,,1.147,2.231,3.36,5.043,12.534
4,0.09,0.104,0.117,0.138,0.218,,1.035,1.842,2.685,3.95,9.705
Big,0.099,0.106,0.116,0.135,0.185,,0.83,1.402,1.943,2.754,5.84


### Regressions

In [189]:
result3a=regCAPM(rVarPortXs,F2)
# List of tuples
regcapmdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ]
# Create DataFrame object from a list of tuples
table4a = pd.DataFrame(regcapmdata, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table4a.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)

,Low,2,3,4,High,,Low,2,3,4,High
,,,a,,,,,,t(a),,
small,0.335,0.337,0.123,-0.158,-1.226,,5.071,4.697,1.616,-1.596,-7.609
2,0.261,0.218,0.202,0.043,-0.719,,4.069,3.245,2.793,0.559,-7.112
3,0.151,0.18,0.14,0.095,-0.439,,2.27,2.624,1.937,1.221,-4.622
4,0.152,0.118,0.042,0.045,-0.277,,2.012,1.62,0.578,0.583,-2.705
Big,0.077,0.098,-0,-0.059,-0.059,,1.342,1.821,-0.006,-1.031,-0.637


In [190]:
result3a=regFivefactor(rVarPortXs,test) #For five factor regression
# List of tuples
reg5Fdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ('', '  ' , 'b' , ' ',' ',' ',' ',' ','t(b)',' ', ' '),
             (result3a[1][0], result3a[1][1], result3a[1][2] , result3a[1][3],result3a[1][4],' ',result3a[3][0], result3a[3][1], result3a[3][2] , result3a[3][3],result3a[3][4]) ,
             (result3a[1][5], result3a[1][6], result3a[1][7] , result3a[1][8],result3a[1][9],' ',result3a[3][5], result3a[3][6], result3a[3][7] , result3a[3][8],result3a[3][9]) ,
             (result3a[1][10], result3a[1][11], result3a[1][12] , result3a[1][13],result3a[1][14],' ',result3a[3][10], result3a[3][11], result3a[3][12] , result3a[3][13],result3a[3][14]) ,
             (result3a[1][15], result3a[1][16], result3a[1][17] , result3a[1][18],result3a[1][19],' ',result3a[3][15], result3a[3][16], result3a[3][17] , result3a[3][18],result3a[3][19]) ,
             (result3a[1][20], result3a[1][21], result3a[1][22] , result3a[1][23],result3a[1][24],' ',result3a[3][20], result3a[3][21], result3a[3][22] , result3a[3][23],result3a[3][24]),
             ('', '  ' , 's' , ' ',' ',' ',' ',' ','t(s)',' ', ' '),
             (result3a[4][0], result3a[4][1], result3a[4][2] , result3a[4][3],result3a[4][4],' ',result3a[5][0], result3a[5][1], result3a[5][2] , result3a[5][3],result3a[5][4]),
             (result3a[4][5], result3a[4][6], result3a[4][7] , result3a[4][8],result3a[4][9],' ',result3a[5][5], result3a[5][6], result3a[5][7] , result3a[5][8],result3a[5][9]),
             (result3a[4][10], result3a[4][11], result3a[4][12] , result3a[4][13],result3a[4][14],' ',result3a[5][10], result3a[5][11], result3a[5][12] , result3a[5][13],result3a[5][14]),
             (result3a[4][15], result3a[4][16], result3a[4][17] , result3a[4][18],result3a[4][19],' ',result3a[5][15], result3a[5][16], result3a[5][17] , result3a[5][18],result3a[5][19]),
             (result3a[4][20], result3a[4][21], result3a[4][22] , result3a[4][23],result3a[4][24],' ',result3a[5][20], result3a[5][21], result3a[5][22] , result3a[5][23],result3a[5][24]),
             ('', '  ' , 'h' , ' ',' ',' ',' ',' ','t(h)',' ', ' '),
             (result3a[6][0], result3a[6][1], result3a[6][2] , result3a[6][3],result3a[6][4],' ',result3a[7][0], result3a[7][1], result3a[7][2] , result3a[7][3],result3a[7][4]) ,
             (result3a[6][5], result3a[6][6], result3a[6][7] , result3a[6][8],result3a[6][9],' ',result3a[7][5], result3a[7][6], result3a[7][7] , result3a[7][8],result3a[7][9]) ,
             (result3a[6][10], result3a[6][11], result3a[6][12] , result3a[6][13],result3a[6][14],' ',result3a[7][10], result3a[7][11], result3a[7][12] , result3a[7][13],result3a[7][14]) ,
             (result3a[6][15], result3a[6][16], result3a[6][17] , result3a[6][18],result3a[6][19],' ',result3a[7][15], result3a[7][16], result3a[7][17] , result3a[7][18],result3a[7][19]) ,
             (result3a[6][20], result3a[6][21], result3a[6][22] , result3a[6][23],result3a[6][24],' ',result3a[7][20], result3a[7][21], result3a[7][22] , result3a[7][23],result3a[7][24]), 
             ('', '  ' , 'r' , ' ',' ',' ',' ',' ','t(r)',' ', ' '),
             (result3a[8][0], result3a[8][1], result3a[8][2] , result3a[8][3],result3a[8][4],' ',result3a[9][0], result3a[9][1], result3a[9][2] , result3a[9][3],result3a[9][4]) ,
             (result3a[8][5], result3a[8][6], result3a[8][7] , result3a[8][8],result3a[8][9],' ',result3a[9][5], result3a[9][6], result3a[9][7] , result3a[9][8],result3a[9][9]) ,
             (result3a[8][10], result3a[8][11], result3a[8][12] , result3a[8][13],result3a[8][14],' ',result3a[9][10], result3a[9][11], result3a[9][12] , result3a[9][13],result3a[9][14]) ,
             (result3a[8][15], result3a[8][16], result3a[8][17] , result3a[8][18],result3a[8][19],' ',result3a[9][15], result3a[9][16], result3a[9][17] , result3a[9][18],result3a[9][19]) ,
             (result3a[8][20], result3a[8][21], result3a[8][22] , result3a[8][23],result3a[8][24],' ',result3a[9][20], result3a[9][21], result3a[9][22] , result3a[9][23],result3a[9][24]),
             ('', '  ' , 'c' , ' ',' ',' ',' ',' ','t(c)',' ', ' '),
             (result3a[10][0], result3a[10][1], result3a[10][2] , result3a[10][3],result3a[10][4],' ',result3a[11][0], result3a[11][1], result3a[11][2] , result3a[11][3],result3a[11][4]) ,
             (result3a[10][5], result3a[10][6], result3a[10][7] , result3a[10][8],result3a[10][9],' ',result3a[11][5], result3a[11][6], result3a[11][7] , result3a[11][8],result3a[11][9]) ,
             (result3a[10][10], result3a[10][11], result3a[10][12] , result3a[10][13],result3a[10][14],' ',result3a[11][10], result3a[11][11], result3a[11][12] , result3a[11][13],result3a[11][14]) ,
             (result3a[10][15], result3a[10][16], result3a[10][17] , result3a[10][18],result3a[10][19],' ',result3a[11][15], result3a[11][16], result3a[11][17] , result3a[11][18],result3a[11][19]) ,
             (result3a[10][20], result3a[10][21], result3a[10][22] , result3a[10][23],result3a[10][24],' ',result3a[11][20], result3a[11][21], result3a[11][22] , result3a[11][23],result3a[11][24])]



# Create DataFrame object from a list of tuples
table4b = pd.DataFrame(reg5Fdata, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big'])
table4b.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)

,Low,2,3,4,High,,Low,2,3,4,High
,,,a,,,,,,t(a),,
small,0.216,0.194,0.062,-0.072,-0.841,,3.394,2.871,0.807,-0.714,-5.62
2,0.113,0.037,0.031,-0.067,-0.456,,1.89,0.619,0.474,-0.918,-4.882
3,0.005,0.017,-0.039,-0.034,-0.216,,0.072,0.28,-0.6,-0.463,-2.41
4,-0.004,-0.062,-0.127,-0.071,-0.034,,-0.055,-0.941,-1.891,-0.939,-0.354
Big,-0.004,-0.039,-0.117,-0.05,0.159,,-0.071,-0.771,-2.208,-0.845,1.8
,,,b,,,,,,t(b),,
small,0.722,0.986,1.1,1.151,1.115,,46.419,59.889,58.644,46.607,30.537
2,0.783,1.009,1.124,1.245,1.257,,53.62,70.084,70.456,69.782,55.176
3,0.787,1,1.102,1.206,1.248,,51.126,67.448,70.455,66.844,57.138


## 6. Accrual 

In [191]:
dt3=acPortXs.mean()
dt3
dt3sd=acPortXs.std()
dt3sd
# List of tuples
students = [ (' ', '  ' , 'Mean' , ' ',' ',' ',' ',' ','SD',' ', ' '),
             (dt3[0], dt3[1], dt3[2] , dt3[3],dt3[4],' ',dt3sd[0], dt3sd[1], dt3sd[2] , dt3sd[3],dt3sd[4]) ,
             (dt3[5], dt3[6], dt3[7] , dt3[8],dt3[9],' ',dt3sd[5], dt3sd[6], dt3sd[7] , dt3sd[8],dt3sd[9]) ,
             (dt3[10], dt3[11], dt3[12] , dt3[13],dt3[14],' ',dt3sd[10], dt3sd[11], dt3sd[12] , dt3sd[13],dt3sd[14]) ,
             (dt3[15], dt3[16], dt3[17] , dt3[18],dt3[19],' ',dt3sd[15], dt3sd[16], dt3sd[17] , dt3sd[18],dt3sd[19]) ,
             (dt3[20], dt3[21], dt3[22] , dt3[23],dt3[24],' ',dt3sd[20], dt3sd[21], dt3sd[22] , dt3sd[23],dt3sd[24]) ]
# Create DataFrame object from a list of tuples
table3 = pd.DataFrame(students, columns = ['Low ' , '2', '3' , '4','High ',' ','Low ' , '2', '3' , '4','High '], index=['','small', '2', '3' , '4' , 'Big'])
table3.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)#Pnael a )

,Low,2,3,4,High,,Low,2,3,4,High
,,,Mean,,,,,,SD,,
small,0.879,0.917,0.831,0.961,0.586,,7.11,6.555,6.204,6.467,7.232
2,0.823,0.88,0.809,0.738,0.631,,6.554,5.804,5.874,6.162,6.801
3,0.872,0.785,0.836,0.79,0.6,,6.203,5.486,5.217,5.661,6.626
4,0.739,0.691,0.656,0.766,0.705,,5.691,5.112,4.979,5.202,6.205
Big,0.623,0.526,0.503,0.492,0.271,,5.082,4.227,4.051,4.549,5.188


In [192]:
#B/M=6/7 OP= 10 INV=11 AC= 9
dt3a=acPort[6].mean()
dt3b=acPort[10].mean()
# List of tuples
students = [ (' ', '  ' , 'B/M' , ' ',' ',' ',' ',' ','OP',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4]) ,
             (dt3a[5], dt3a[6], dt3a[7] , dt3a[8],dt3a[9],' ',dt3b[5], dt3b[6], dt3b[7] , dt3b[8],dt3b[9]) ,
             (dt3a[10], dt3a[11], dt3a[12] , dt3a[13],dt3a[14],' ',dt3b[10], dt3b[11], dt3b[12] , dt3b[13],dt3b[14]) ,
             (dt3a[15], dt3a[16], dt3a[17] , dt3a[18],dt3a[19],' ',dt3b[15],dt3b[16], dt3b[17] , dt3b[18],dt3b[19]) ,
             (dt3a[20], dt3a[21], dt3a[22] , dt3a[23],dt3a[24],' ',dt3b[20], dt3b[21], dt3b[22] , dt3b[23],dt3b[24]) ]
# Create DataFrame object from a list of tuples
table3a = pd.DataFrame(students, columns = ['Low ' , '2', '3' , '4','High ',' ','Low ' , '2', '3' , '4','High '], index=['','small', '2', '3' , '4' , 'Big'])
table3a.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)#Pnael a 



,Low,2,3,4,High,,Low,2,3,4,High
,,,B/M,,,,,,OP,,
small,0.909,0.969,0.974,0.904,0.757,,-0.036,0.156,0.162,0.186,0.59
2,0.738,0.828,0.771,0.717,0.617,,0.233,0.229,0.237,0.282,0.381
3,0.689,0.74,0.744,0.656,0.548,,0.34,0.259,0.265,0.288,0.399
4,0.632,0.717,0.7,0.603,0.491,,0.381,0.283,0.283,0.312,0.49
Big,0.505,0.562,0.536,0.451,0.397,,0.451,0.352,0.349,0.383,0.734


In [193]:
#B/M=6/7 OP= 10 INV=11 AC= 9
dt3a=acPort[11].mean()
dt3b=acPort[9].mean()
# List of tuples
students = [ (' ', '  ' , 'Inv' , ' ',' ',' ',' ',' ','AC',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4]) ,
             (dt3a[5], dt3a[6], dt3a[7] , dt3a[8],dt3a[9],' ',dt3b[5], dt3b[6], dt3b[7] , dt3b[8],dt3b[9]) ,
             (dt3a[10], dt3a[11], dt3a[12] , dt3a[13],dt3a[14],' ',dt3b[10], dt3b[11], dt3b[12] , dt3b[13],dt3b[14]) ,
             (dt3a[15], dt3a[16], dt3a[17] , dt3a[18],dt3a[19],' ',dt3b[15],dt3b[16], dt3b[17] , dt3b[18],dt3b[19]) ,
             (dt3a[20], dt3a[21], dt3a[22] , dt3a[23],dt3a[24],' ',dt3b[20], dt3b[21], dt3b[22] , dt3b[23],dt3b[24]) ]
# Create DataFrame object from a list of tuples
table3b = pd.DataFrame(students, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table3b.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)#Pnael a 

,Low,2,3,4,High,,Low,2,3,4,High
,,,Inv,,,,,,AC,,
small,0.08,0.128,0.141,0.158,0.26,,-63.27,-2.423,1.732,6.313,59.332
2,0.127,0.141,0.155,0.164,0.269,,-42.328,-2.344,1.72,6.25,46.154
3,0.132,0.139,0.138,0.16,0.26,,-94.727,-2.329,1.723,6.178,40.551
4,0.111,0.114,0.123,0.142,0.222,,-24.171,-2.305,1.668,6.102,30.454
Big,0.127,0.111,0.11,0.124,0.176,,-17.067,-2.282,1.634,5.904,30.256


### Regressions

### Panel A: Regression Intercepts

#### Three Factor 

In [194]:
result3a=regCAPM(acPortXs,F2) #MKT,SMB,HML
# List of tuples
regcapmdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ]
# Create DataFrame object from a list of tuples
table4a = pd.DataFrame(regcapmdata, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table4a.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)

,Low,2,3,4,High,,Low,2,3,4,High
,,,a,,,,,,t(a),,
small,0.013,0.127,0.064,0.162,-0.284,,0.163,1.673,0.933,2.534,-4.135
2,0.01,0.146,0.082,-0.043,-0.161,,0.17,2.334,1.296,-0.699,-2.617
3,0.155,0.136,0.185,0.073,-0.171,,1.984,1.953,2.812,1.131,-2.084
4,0.096,0.068,0.019,0.167,0.027,,1.205,0.999,0.289,2.512,0.331
Big,0.211,0.167,0.112,0.066,-0.157,,2.673,3.016,2.028,1.073,-1.782


#### Four Factor

In [195]:
result3a=regFourFactor(acPortXs,F4) #MKT,SMB,HML,CMA
# List of tuples
regcapmdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ]
# Create DataFrame object from a list of tuples
table4a = pd.DataFrame(regcapmdata, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table4a.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)

,Low,2,3,4,High,,Low,2,3,4,High
,,,a,,,,,,t(a),,
small,-0.034,0.076,0.021,0.145,-0.279,,-0.434,1.005,0.312,2.251,-4.01
2,-0.008,0.123,0.048,-0.045,-0.134,,-0.133,1.957,0.767,-0.732,-2.171
3,0.16,0.104,0.16,0.087,-0.106,,2.035,1.493,2.416,1.336,-1.308
4,0.069,0.024,0.017,0.146,0.062,,0.857,0.36,0.254,2.188,0.743
Big,0.208,0.156,0.081,0.091,-0.1,,2.608,2.792,1.46,1.464,-1.148


#### Five Factor

In [196]:
result3a=regCAPM(acPortXs,test) #MKT,SMB,HMLO,RMW,CMA
# List of tuples
regcapmdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ]
# Create DataFrame object from a list of tuples
table4a = pd.DataFrame(regcapmdata, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table4a.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)

,Low,2,3,4,High,,Low,2,3,4,High
,,,a,,,,,,t(a),,
small,0.11,0.256,0.143,0.226,-0.184,,1.49,3.797,2.199,3.537,-2.695
2,0.038,0.139,0.094,-0.065,-0.154,,0.61,2.168,1.483,-1.031,-2.446
3,0.193,0.162,0.189,0.044,-0.197,,2.405,2.302,2.812,0.674,-2.459
4,0.063,0.086,-0.008,0.137,0.038,,0.77,1.267,-0.115,2.012,0.45
Big,0.23,0.207,0.019,0.026,-0.183,,2.837,3.704,0.345,0.416,-2.093


### Panel B: Regression slopes

#### 3 Factors

In [197]:
result3a=regCAPM(acPortXs,F2) #MKT,SMB,HML
# List of tuples
regcapmdata = [ ('', '  ' , 'h' , ' ',' ',' ',' ',' ','t(h)',' ', ' '),
             (result3a[6][0], result3a[6][1], result3a[6][2] , result3a[6][3],result3a[6][4],' ',result3a[7][0], result3a[7][1], result3a[7][2] , result3a[7][3],result3a[7][4]) ,
             (result3a[6][5], result3a[6][6], result3a[6][7] , result3a[6][8],result3a[6][9],' ',result3a[7][5], result3a[7][6], result3a[7][7] , result3a[7][8],result3a[7][9]) ,
             (result3a[6][10], result3a[6][11], result3a[6][12] , result3a[6][13],result3a[6][14],' ',result3a[7][10], result3a[7][11], result3a[7][12] , result3a[7][13],result3a[7][14]) ,
             (result3a[6][15], result3a[6][16], result3a[6][17] , result3a[6][18],result3a[6][19],' ',result3a[7][15], result3a[7][16], result3a[7][17] , result3a[7][18],result3a[7][19]) ,
             (result3a[6][20], result3a[6][21], result3a[6][22] , result3a[6][23],result3a[6][24],' ',result3a[7][20], result3a[7][21], result3a[7][22] , result3a[7][23],result3a[7][24]),
             ]
# Create DataFrame object from a list of tuples
table4a = pd.DataFrame(regcapmdata, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table4a.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)

,Low,2,3,4,High,,Low,2,3,4,High
,,,h,,,,,,t(h),,
small,-0.003,-0.012,0.022,0.009,-0.043,,-0.107,-0.426,0.903,0.394,-1.713
2,-0.002,0.044,0.01,0.039,-0.113,,-0.084,1.939,0.429,1.752,-5.061
3,-0.06,-0.023,0.045,0.066,-0.065,,-2.144,-0.925,1.879,2.851,-2.189
4,-0.046,0.076,0.142,-0.023,-0.114,,-1.587,3.082,5.924,-0.948,-3.836
Big,-0.211,-0.101,0.001,-0.084,-0.2,,-7.406,-5.076,0.057,-3.772,-6.299


#### 4 Factors

In [198]:
result3a=regFourFactor(acPortXs,F4) #For five factor regression
# List of tuples
reg5Fdata = [ ('', '  ' , 'h' , ' ',' ',' ',' ',' ','t(h)',' ', ' '),
             (result3a[6][0], result3a[6][1], result3a[6][2] , result3a[6][3],result3a[6][4],' ',result3a[7][0], result3a[7][1], result3a[7][2] , result3a[7][3],result3a[7][4]) ,
             (result3a[6][5], result3a[6][6], result3a[6][7] , result3a[6][8],result3a[6][9],' ',result3a[7][5], result3a[7][6], result3a[7][7] , result3a[7][8],result3a[7][9]) ,
             (result3a[6][10], result3a[6][11], result3a[6][12] , result3a[6][13],result3a[6][14],' ',result3a[7][10], result3a[7][11], result3a[7][12] , result3a[7][13],result3a[7][14]) ,
             (result3a[6][15], result3a[6][16], result3a[6][17] , result3a[6][18],result3a[6][19],' ',result3a[7][15], result3a[7][16], result3a[7][17] , result3a[7][18],result3a[7][19]) ,
             (result3a[6][20], result3a[6][21], result3a[6][22] , result3a[6][23],result3a[6][24],' ',result3a[7][20], result3a[7][21], result3a[7][22] , result3a[7][23],result3a[7][24]), 
             ('', '  ' , 'c' , ' ',' ',' ',' ',' ','t(c)',' ', ' '),
             (result3a[8][0], result3a[8][1], result3a[8][2] , result3a[8][3],result3a[8][4],' ',result3a[9][0], result3a[9][1], result3a[9][2] , result3a[9][3],result3a[9][4]) ,
             (result3a[8][5], result3a[8][6], result3a[8][7] , result3a[8][8],result3a[8][9],' ',result3a[9][5], result3a[9][6], result3a[9][7] , result3a[9][8],result3a[9][9]) ,
             (result3a[8][10], result3a[8][11], result3a[8][12] , result3a[8][13],result3a[8][14],' ',result3a[9][10], result3a[9][11], result3a[9][12] , result3a[9][13],result3a[9][14]) ,
             (result3a[8][15], result3a[8][16], result3a[8][17] , result3a[8][18],result3a[8][19],' ',result3a[9][15], result3a[9][16], result3a[9][17] , result3a[9][18],result3a[9][19]) ,
             (result3a[8][20], result3a[8][21], result3a[8][22] , result3a[8][23],result3a[8][24],' ',result3a[9][20], result3a[9][21], result3a[9][22] , result3a[9][23],result3a[9][24])]
            


# Create DataFrame object from a list of tuples
table4b = pd.DataFrame(reg5Fdata, columns = ['Low β' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big'])
table4b.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)

,Low β,2,3,4,High,,Low,2,3,4,High
,,,h,,,,,,t(h),,
small,-0.111,-0.131,-0.077,-0.03,-0.029,,-0.111,-0.131,-0.077,-0.03,-0.029
2,-0.045,-0.01,-0.068,0.033,-0.05,,-0.045,-0.01,-0.068,0.033,-0.05
3,-0.047,-0.098,-0.014,0.099,0.087,,-0.047,-0.098,-0.014,0.099,0.087
4,-0.109,-0.026,0.137,-0.071,-0.035,,-0.109,-0.026,0.137,-0.071,-0.035
Big,-0.217,-0.127,-0.073,-0.027,-0.069,,-0.217,-0.127,-0.073,-0.027,-0.069
,,,c,,,,,,t(c),,
small,0.235,0.26,0.216,0.086,-0.029,,4.298,4.9,4.503,1.895,-0.593
2,0.094,0.116,0.169,0.012,-0.137,,2.167,2.624,3.812,0.286,-3.154
3,-0.028,0.161,0.128,-0.071,-0.331,,-0.513,3.284,2.754,-1.541,-5.828


#### 5 Factors

In [199]:
result3a=regFivefactor(acPortXs,test) #For five factor regression
# List of tuples
reg5Fdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ('', '  ' , 'b' , ' ',' ',' ',' ',' ','t(b)',' ', ' '),
             (result3a[1][0], result3a[1][1], result3a[1][2] , result3a[1][3],result3a[1][4],' ',result3a[3][0], result3a[3][1], result3a[3][2] , result3a[3][3],result3a[3][4]) ,
             (result3a[1][5], result3a[1][6], result3a[1][7] , result3a[1][8],result3a[1][9],' ',result3a[3][5], result3a[3][6], result3a[3][7] , result3a[3][8],result3a[3][9]) ,
             (result3a[1][10], result3a[1][11], result3a[1][12] , result3a[1][13],result3a[1][14],' ',result3a[3][10], result3a[3][11], result3a[3][12] , result3a[3][13],result3a[3][14]) ,
             (result3a[1][15], result3a[1][16], result3a[1][17] , result3a[1][18],result3a[1][19],' ',result3a[3][15], result3a[3][16], result3a[3][17] , result3a[3][18],result3a[3][19]) ,
             (result3a[1][20], result3a[1][21], result3a[1][22] , result3a[1][23],result3a[1][24],' ',result3a[3][20], result3a[3][21], result3a[3][22] , result3a[3][23],result3a[3][24]),
             ('', '  ' , 's' , ' ',' ',' ',' ',' ','t(s)',' ', ' '),
             (result3a[4][0], result3a[4][1], result3a[4][2] , result3a[4][3],result3a[4][4],' ',result3a[5][0], result3a[5][1], result3a[5][2] , result3a[5][3],result3a[5][4]),
             (result3a[4][5], result3a[4][6], result3a[4][7] , result3a[4][8],result3a[4][9],' ',result3a[5][5], result3a[5][6], result3a[5][7] , result3a[5][8],result3a[5][9]),
             (result3a[4][10], result3a[4][11], result3a[4][12] , result3a[4][13],result3a[4][14],' ',result3a[5][10], result3a[5][11], result3a[5][12] , result3a[5][13],result3a[5][14]),
             (result3a[4][15], result3a[4][16], result3a[4][17] , result3a[4][18],result3a[4][19],' ',result3a[5][15], result3a[5][16], result3a[5][17] , result3a[5][18],result3a[5][19]),
             (result3a[4][20], result3a[4][21], result3a[4][22] , result3a[4][23],result3a[4][24],' ',result3a[5][20], result3a[5][21], result3a[5][22] , result3a[5][23],result3a[5][24]),
             ('', '  ' , 'h' , ' ',' ',' ',' ',' ','t(h)',' ', ' '),
             (result3a[6][0], result3a[6][1], result3a[6][2] , result3a[6][3],result3a[6][4],' ',result3a[7][0], result3a[7][1], result3a[7][2] , result3a[7][3],result3a[7][4]) ,
             (result3a[6][5], result3a[6][6], result3a[6][7] , result3a[6][8],result3a[6][9],' ',result3a[7][5], result3a[7][6], result3a[7][7] , result3a[7][8],result3a[7][9]) ,
             (result3a[6][10], result3a[6][11], result3a[6][12] , result3a[6][13],result3a[6][14],' ',result3a[7][10], result3a[7][11], result3a[7][12] , result3a[7][13],result3a[7][14]) ,
             (result3a[6][15], result3a[6][16], result3a[6][17] , result3a[6][18],result3a[6][19],' ',result3a[7][15], result3a[7][16], result3a[7][17] , result3a[7][18],result3a[7][19]) ,
             (result3a[6][20], result3a[6][21], result3a[6][22] , result3a[6][23],result3a[6][24],' ',result3a[7][20], result3a[7][21], result3a[7][22] , result3a[7][23],result3a[7][24]), 
             ('', '  ' , 'r' , ' ',' ',' ',' ',' ','t(r)',' ', ' '),
             (result3a[8][0], result3a[8][1], result3a[8][2] , result3a[8][3],result3a[8][4],' ',result3a[9][0], result3a[9][1], result3a[9][2] , result3a[9][3],result3a[9][4]) ,
             (result3a[8][5], result3a[8][6], result3a[8][7] , result3a[8][8],result3a[8][9],' ',result3a[9][5], result3a[9][6], result3a[9][7] , result3a[9][8],result3a[9][9]) ,
             (result3a[8][10], result3a[8][11], result3a[8][12] , result3a[8][13],result3a[8][14],' ',result3a[9][10], result3a[9][11], result3a[9][12] , result3a[9][13],result3a[9][14]) ,
             (result3a[8][15], result3a[8][16], result3a[8][17] , result3a[8][18],result3a[8][19],' ',result3a[9][15], result3a[9][16], result3a[9][17] , result3a[9][18],result3a[9][19]) ,
             (result3a[8][20], result3a[8][21], result3a[8][22] , result3a[8][23],result3a[8][24],' ',result3a[9][20], result3a[9][21], result3a[9][22] , result3a[9][23],result3a[9][24]),
             ('', '  ' , 'c' , ' ',' ',' ',' ',' ','t(c)',' ', ' '),
             (result3a[10][0], result3a[10][1], result3a[10][2] , result3a[10][3],result3a[10][4],' ',result3a[11][0], result3a[11][1], result3a[11][2] , result3a[11][3],result3a[11][4]) ,
             (result3a[10][5], result3a[10][6], result3a[10][7] , result3a[10][8],result3a[10][9],' ',result3a[11][5], result3a[11][6], result3a[11][7] , result3a[11][8],result3a[11][9]) ,
             (result3a[10][10], result3a[10][11], result3a[10][12] , result3a[10][13],result3a[10][14],' ',result3a[11][10], result3a[11][11], result3a[11][12] , result3a[11][13],result3a[11][14]) ,
             (result3a[10][15], result3a[10][16], result3a[10][17] , result3a[10][18],result3a[10][19],' ',result3a[11][15], result3a[11][16], result3a[11][17] , result3a[11][18],result3a[11][19]) ,
             (result3a[10][20], result3a[10][21], result3a[10][22] , result3a[10][23],result3a[10][24],' ',result3a[11][20], result3a[11][21], result3a[11][22] , result3a[11][23],result3a[11][24])]



# Create DataFrame object from a list of tuples
table4b = pd.DataFrame(reg5Fdata, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big'])
table4b.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)

,Low,2,3,4,High,,Low,2,3,4,High
,,,a,,,,,,t(a),,
small,0.11,0.256,0.143,0.226,-0.184,,1.49,3.797,2.199,3.537,-2.695
2,0.038,0.139,0.094,-0.065,-0.154,,0.61,2.168,1.483,-1.031,-2.446
3,0.193,0.162,0.189,0.044,-0.197,,2.405,2.302,2.812,0.674,-2.459
4,0.063,0.086,-0.008,0.137,0.038,,0.77,1.267,-0.115,2.012,0.45
Big,0.23,0.207,0.019,0.026,-0.183,,2.837,3.704,0.345,0.416,-2.093
,,,b,,,,,,t(b),,
small,1.021,0.925,0.902,0.936,1.026,,56.897,56.227,56.916,60.14,61.714
2,1.104,1.01,0.968,1.015,1.109,,72.677,64.833,62.357,66.327,72.289
3,1.112,0.99,0.935,1.065,1.16,,56.955,57.728,57.063,66.283,59.323


## 7.Momentum

In [200]:
dt3=momPortXs.mean()
dt3
dt3sd=momPortXs.std()
dt3sd
# List of tuples
students = [ (' ', '  ' , 'Mean' , ' ',' ',' ',' ',' ','SD',' ', ' '),
             (dt3[0], dt3[1], dt3[2] , dt3[3],dt3[4],' ',dt3sd[0], dt3sd[1], dt3sd[2] , dt3sd[3],dt3sd[4]) ,
             (dt3[5], dt3[6], dt3[7] , dt3[8],dt3[9],' ',dt3sd[5], dt3sd[6], dt3sd[7] , dt3sd[8],dt3sd[9]) ,
             (dt3[10], dt3[11], dt3[12] , dt3[13],dt3[14],' ',dt3sd[10], dt3sd[11], dt3sd[12] , dt3sd[13],dt3sd[14]) ,
             (dt3[15], dt3[16], dt3[17] , dt3[18],dt3[19],' ',dt3sd[15], dt3sd[16], dt3sd[17] , dt3sd[18],dt3sd[19]) ,
             (dt3[20], dt3[21], dt3[22] , dt3[23],dt3[24],' ',dt3sd[20], dt3sd[21], dt3sd[22] , dt3sd[23],dt3sd[24]) ]
# Create DataFrame object from a list of tuples
table3 = pd.DataFrame(students, columns = ['Low ' , '2', '3' , '4','High ',' ','Low ' , '2', '3' , '4','High '], index=['','small', '2', '3' , '4' , 'Big'])
table3.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)#Pnael a )

,Low,2,3,4,High,,Low,2,3,4,High
,,,Mean,,,,,,SD,,
small,0.029,0.674,0.918,1.048,1.379,,8.024,5.88,5.435,5.502,6.78
2,0.137,0.663,0.812,1.01,1.222,,7.836,5.872,5.267,5.404,6.729
3,0.266,0.613,0.712,0.782,1.182,,7.36,5.517,5.065,5.001,6.314
4,0.199,0.595,0.651,0.794,1.043,,7.261,5.52,4.875,4.794,5.893
Big,0.165,0.459,0.398,0.55,0.783,,6.79,4.882,4.371,4.313,5.266


### No data for B/M, OP INV and AC

In [201]:
#B/M=6/7 OP= 10 INV=11 AC= 9
dt3a=momPort[6].mean()
dt3b=momPort[10].mean()
# List of tuples
students = [ (' ', '  ' , 'B/M' , ' ',' ',' ',' ',' ','OP',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4]) ,
             (dt3a[5], dt3a[6], dt3a[7] , dt3a[8],dt3a[9],' ',dt3b[5], dt3b[6], dt3b[7] , dt3b[8],dt3b[9]) ,
             (dt3a[10], dt3a[11], dt3a[12] , dt3a[13],dt3a[14],' ',dt3b[10], dt3b[11], dt3b[12] , dt3b[13],dt3b[14]) ,
             (dt3a[15], dt3a[16], dt3a[17] , dt3a[18],dt3a[19],' ',dt3b[15],dt3b[16], dt3b[17] , dt3b[18],dt3b[19]) ,
             (dt3a[20], dt3a[21], dt3a[22] , dt3a[23],dt3a[24],' ',dt3b[20], dt3b[21], dt3b[22] , dt3b[23],dt3b[24]) ]
# Create DataFrame object from a list of tuples
table3a = pd.DataFrame(students, columns = ['Low ' , '2', '3' , '4','High ',' ','Low ' , '2', '3' , '4','High '], index=['','small', '2', '3' , '4' , 'Big'])
table3a#Pnael a 




KeyError: 10

In [ ]:
#B/M=6/7 OP= 10 INV=11 AC= 9
dt3a=momPort[11].mean()
dt3b=momPort[6].mean()
# List of tuples
students = [ (' ', '  ' , 'Inv' , ' ',' ',' ',' ',' ','Prior2-12',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4]) ,
             (dt3a[5], dt3a[6], dt3a[7] , dt3a[8],dt3a[9],' ',dt3b[5], dt3b[6], dt3b[7] , dt3b[8],dt3b[9]) ,
             (dt3a[10], dt3a[11], dt3a[12] , dt3a[13],dt3a[14],' ',dt3b[10], dt3b[11], dt3b[12] , dt3b[13],dt3b[14]) ,
             (dt3a[15], dt3a[16], dt3a[17] , dt3a[18],dt3a[19],' ',dt3b[15],dt3b[16], dt3b[17] , dt3b[18],dt3b[19]) ,
             (dt3a[20], dt3a[21], dt3a[22] , dt3a[23],dt3a[24],' ',dt3b[20], dt3b[21], dt3b[22] , dt3b[23],dt3b[24]) ]
# Create DataFrame object from a list of tuples
table3b = pd.DataFrame(students, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table3b#Pnael a 

### Regression 

#### 5 Factors

In [202]:
result3a=regFivefactor(momPortXs,F6)
# List of tuples
regcapmdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ]
# Create DataFrame object from a list of tuples
table4a = pd.DataFrame(regcapmdata, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table4a.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)

,Low,2,3,4,High,,Low,2,3,4,High
,,,a,,,,,,t(a),,
small,-0.713,-0.197,0.039,0.216,0.622,,-4.696,-2.5,0.619,3.287,6.488
2,-0.62,-0.209,-0.045,0.153,0.511,,-4.408,-2.566,-0.788,2.854,5.697
3,-0.372,-0.19,-0.126,-0.07,0.51,,-2.415,-2.248,-1.981,-1.14,5.265
4,-0.419,-0.21,-0.147,0.012,0.429,,-2.502,-2.17,-2.188,0.185,4.015
Big,-0.375,-0.133,-0.183,-0.043,0.318,,-2.28,-1.313,-2.848,-0.673,2.955


#### 6 Factors

In [203]:
momtest=test.copy()
momtest['Mom']=FF6['Mom   ']

In [204]:
momtest

,Mkt-RF,SMB,HMLO,RMW,CMA,Mom
Date,,,,,,
1963-07,-0.39,-0.47,0.243781,0.66,-1.15,1.00
1963-08,5.07,-0.79,1.945587,0.40,-0.40,1.03
1963-09,-1.57,-0.48,0.093831,-0.76,0.24,0.16
1963-10,2.53,-1.29,1.723674,2.75,-2.24,3.14
1963-11,-0.85,-0.84,-0.413395,-0.45,2.22,-0.75
...,...,...,...,...,...,...
2014-08,4.24,0.30,0.134607,-0.61,-0.70,0.79
2014-09,-1.97,-3.75,-0.831468,1.24,-0.53,0.44
2014-10,2.52,3.80,-1.701555,-0.48,-0.19,-0.04


In [205]:
result3a=regSixfactor(momPortXs,momtest)
result3a
# List of tuples
reg5Fdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ('', '  ' , 'h' , ' ',' ',' ',' ',' ','t(h)',' ', ' '),
             (result3a[6][0], result3a[6][1], result3a[6][2] , result3a[6][3],result3a[6][4],' ',result3a[7][0], result3a[7][1], result3a[7][2] , result3a[7][3],result3a[7][4]) ,
             (result3a[6][5], result3a[6][6], result3a[6][7] , result3a[6][8],result3a[6][9],' ',result3a[7][5], result3a[7][6], result3a[7][7] , result3a[7][8],result3a[7][9]) ,
             (result3a[6][10], result3a[6][11], result3a[6][12] , result3a[6][13],result3a[6][14],' ',result3a[7][10], result3a[7][11], result3a[7][12] , result3a[7][13],result3a[7][14]) ,
             (result3a[6][15], result3a[6][16], result3a[6][17] , result3a[6][18],result3a[6][19],' ',result3a[7][15], result3a[7][16], result3a[7][17] , result3a[7][18],result3a[7][19]) ,
             (result3a[6][20], result3a[6][21], result3a[6][22] , result3a[6][23],result3a[6][24],' ',result3a[7][20], result3a[7][21], result3a[7][22] , result3a[7][23],result3a[7][24]), 
             ('', '  ' , 'r' , ' ',' ',' ',' ',' ','t(r)',' ', ' '),
             (result3a[8][0], result3a[8][1], result3a[8][2] , result3a[8][3],result3a[8][4],' ',result3a[9][0], result3a[9][1], result3a[9][2] , result3a[9][3],result3a[9][4]) ,
             (result3a[8][5], result3a[8][6], result3a[8][7] , result3a[8][8],result3a[8][9],' ',result3a[9][5], result3a[9][6], result3a[9][7] , result3a[9][8],result3a[9][9]) ,
             (result3a[8][10], result3a[8][11], result3a[8][12] , result3a[8][13],result3a[8][14],' ',result3a[9][10], result3a[9][11], result3a[9][12] , result3a[9][13],result3a[9][14]) ,
             (result3a[8][15], result3a[8][16], result3a[8][17] , result3a[8][18],result3a[8][19],' ',result3a[9][15], result3a[9][16], result3a[9][17] , result3a[9][18],result3a[9][19]) ,
             (result3a[8][20], result3a[8][21], result3a[8][22] , result3a[8][23],result3a[8][24],' ',result3a[9][20], result3a[9][21], result3a[9][22] , result3a[9][23],result3a[9][24]),
             ('', '  ' , 'c' , ' ',' ',' ',' ',' ','t(c)',' ', ' '),
             (result3a[10][0], result3a[10][1], result3a[10][2] , result3a[10][3],result3a[10][4],' ',result3a[11][0], result3a[11][1], result3a[11][2] , result3a[11][3],result3a[11][4]) ,
             (result3a[10][5], result3a[10][6], result3a[10][7] , result3a[10][8],result3a[10][9],' ',result3a[11][5], result3a[11][6], result3a[11][7] , result3a[11][8],result3a[11][9]) ,
             (result3a[10][10], result3a[10][11], result3a[10][12] , result3a[10][13],result3a[10][14],' ',result3a[11][10], result3a[11][11], result3a[11][12] , result3a[11][13],result3a[11][14]) ,
             (result3a[10][15], result3a[10][16], result3a[10][17] , result3a[10][18],result3a[10][19],' ',result3a[11][15], result3a[11][16], result3a[11][17] , result3a[11][18],result3a[11][19]) ,
             (result3a[10][20], result3a[10][21], result3a[10][22] , result3a[10][23],result3a[10][24],' ',result3a[11][20], result3a[11][21], result3a[11][22] , result3a[11][23],result3a[11][24]),
             ('', '  ' , 'm' , ' ',' ',' ',' ',' ','t(m)',' ', ' '),
             (result3a[12][0], result3a[12][1], result3a[12][2] , result3a[12][3],result3a[12][4],' ',result3a[13][0], result3a[13][1], result3a[13][2] , result3a[13][3],result3a[13][4]) ,
             (result3a[12][5], result3a[12][6], result3a[12][7] , result3a[12][8],result3a[12][9],' ',result3a[13][5], result3a[13][6], result3a[13][7] , result3a[13][8],result3a[13][9]) ,
             (result3a[12][10], result3a[12][11], result3a[12][12] , result3a[12][13],result3a[12][14],' ',result3a[13][10], result3a[13][11], result3a[13][12] , result3a[13][13],result3a[13][14]) ,
             (result3a[12][15], result3a[12][16], result3a[12][17] , result3a[12][18],result3a[12][19],' ',result3a[13][15], result3a[13][16], result3a[13][17] , result3a[13][18],result3a[13][19]) ,
             (result3a[12][20], result3a[12][21], result3a[12][22] , result3a[12][23],result3a[12][24],' ',result3a[13][20], result3a[13][21], result3a[13][22] , result3a[13][23],result3a[3][24])]



# Create DataFrame object from a list of tuples
table4b = pd.DataFrame(reg5Fdata, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big'])
table4b
table4b.applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x) #Rounded value to two decimal places

,Low,2,3,4,High,,Low,2,3,4,High
,,,a,,,,,,t(a),,
small,-0.226,-0.004,0.105,0.165,0.402,,-2.285,-0.062,1.698,2.526,4.939
2,-0.115,0.03,0.009,0.104,0.244,,-1.57,0.501,0.159,1.957,3.815
3,0.157,0.053,-0.013,-0.129,0.211,,1.761,0.851,-0.218,-2.115,3.179
4,0.147,0.081,-0.028,-0.032,0.104,,1.465,1.179,-0.456,-0.511,1.397
Big,0.163,0.185,-0.101,-0.153,-0.029,,1.573,2.737,-1.62,-2.596,-0.422
,,,h,,,,,,t(h),,
small,0.106,0.277,0.286,0.213,0.053,,2.205,8.803,9.514,6.709,1.35
2,0.059,0.185,0.24,0.215,0.02,,1.647,6.435,8.709,8.338,0.638
3,0.038,0.187,0.245,0.273,-0.014,,0.882,6.17,8.628,9.25,-0.438
